# 패키지, 데이터 로드

In [1]:
# 패키지 로드 
import pandas as pd 
from bs4 import BeautifulSoup  # html 파싱 
import re  # 정규표현식

# 데이터 로드 
news1 = pd.read_excel('../../../data/News/news_1filtered_202301.xlsx')
news2 = pd.read_excel('../../../data/News/news_1filtered_202302.xlsx')
news3 = pd.read_excel('../../../data/News/news_1filtered_202303.xlsx')
news4 = pd.read_excel('../../../data/News/news_1filtered_202304.xlsx')
news5 = pd.read_excel('../../../data/News/news_1filtered_202305.xlsx')
news6 = pd.read_excel('../../../data/News/news_1filtered_202306.xlsx')
dataframes = [news1, news2, news3, news4, news5, news6] 
news = pd.concat(dataframes, ignore_index=True) # 1~6월 뉴스기사 합치기 
news.head()

,NEWS_YM,DATE_TIME,WRITER,SUBCATEGORY,TITLE,IMPORTANCE,ITEM_NAME,TAG_LIST,CONTENT
0,202301,20230102_135224,머니투데이,기술,"""'통신→디지털' 혁신을 성과로""…SKT·KT·LGU+ 신년 화두는",53.75,SK텔레콤|KT|LG유플러스,AI| 디지털| 통신| 미디어| 글로벌| 혁신| 인터넷| 플랫폼,"[머니투데이 변휘 기자, 김승한 기자] <img src=""http://thumb.m..."
1,202301,20230106_131618,머니투데이,경제,"구찌 티셔츠에 한글 로고, 가방엔 JEJU...""한국 독점 상품""",66.21,NaN,독점| 명품| 소비| 제주| 가방,"[머니투데이 정인지 기자] <img src=""http://thumb.mt.co.kr..."
2,202301,20230111_120209,머니투데이,경제,"YG 트레저, 모바일 게임 '뿌까 퍼즐 어드벤처' 컬래버레이션",46.75,와이지엔터테인먼트,모바일| 게임| 캐릭터,"[머니투데이 김건우 기자] <img src=""http://thumb.mt.co.kr..."
3,202301,20230117_132642,머니투데이,사회,"현대오토에버, 1분기 신입·경력 채용 시작…""MZ가 면접관으로""",60.51,NaN,소프트웨어| 모빌리티| 문화| 교육| 자동차| 디지털,"[머니투데이 정한결 기자] <img src=""http://thumb.mt.co.kr..."
4,202301,20230102_084347,eDaily,사회,BTS 자리 넘본다…유망주는 스트레이 키즈-KB,67.91,NaN,BTS| 판매량| NCT| 넥스트| 엔하이픈| 블랙핑크| 에스파| 아이브,<font color='gray'><b>- KB증권 “군 입대 BTS에 도전 이어질...


In [2]:
print('\n','------------------데이터프레임 기본 정보----------------','\n')
print(news.info())  
print('\n', '-----------------열별 결측치 개수 확인----------------', '\n')
print(news.isna().sum())  
print('\n', '-----------------언론사 고유값 개수 확인----------------', '\n')
print(news['WRITER'].nunique())
print('\n','-----------------언론사 고유값 확인(언론사 종류)----------------', '\n')
print(news['WRITER'].unique())


 ------------------데이터프레임 기본 정보---------------- 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16177 entries, 0 to 16176
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   NEWS_YM      16177 non-null  int64  
 1   DATE_TIME    16177 non-null  object 
 2   WRITER       16177 non-null  object 
 3   SUBCATEGORY  16177 non-null  object 
 4   TITLE        16176 non-null  object 
 5   IMPORTANCE   16177 non-null  float64
 6   ITEM_NAME    6889 non-null   object 
 7   TAG_LIST     11191 non-null  object 
 8   CONTENT      16177 non-null  object 
dtypes: float64(1), int64(1), object(7)
memory usage: 1.1+ MB
None

 -----------------열별 결측치 개수 확인---------------- 

NEWS_YM           0
DATE_TIME         0
WRITER            0
SUBCATEGORY       0
TITLE             1
IMPORTANCE        0
ITEM_NAME      9288
TAG_LIST       4986
CONTENT           0
dtype: int64

 -----------------언론사 고유값 개수 확인---------------- 

31

 -------------

- TITLE 변수가 결측인 행이 하나 존재한다. 
- 아래 코드에서 직접 기사를 확인하도록 한다.

# TITLE(제목)변수가 결측치인 행 처리

In [3]:
# TITLE 변수가 결측치인 행 확인
news[news['TITLE'].isna()]['CONTENT'].iloc[0]

'<div>[프라임경제] 유통업계가 사회적 거리두기 해제 이후 첫 밸런타인데이를 맞아 다양한 상품과 혜택을 준비했다.&nbsp;</div><div><br></div><div>먼저 이마트 밸런타인데이를 앞두고 15일까지 초콜릿 최대 50% 할인 행사와 동시에 구매 금액대별 최대 30% 할인 쿠폰 3종을 추가로 지급한다.</div><div><br></div><div>대면 접촉이 어려웠던 코로나 기간 동안 밸런타인데이 매출은 코로나 이전 대비 약 20% 가량 줄었다.</div><div><br></div><div><img xtype="photo" src="http://www.newsprime.co.kr/data/photos/cdn/20230206/art_592399_1675748594.jpg" class="sm-image-l" title="(C) 이마트 ">그러나 올해는 사회적 거리두기가 해제 되고 처음 맞이하는 만큼 사무실 출근과 학교/학원 출석, 대면 모임 등 지인들과 얼굴을 마주할 자리가 늘어남에 따라 밸런타인데이 초콜릿 구매도 증가 할 것으로 업계는 기대하고 있다</div><div><br></div><div>이마트는 노이하우스 초콜릿 외에도 해외 유명 프리미엄 초콜릿부터 대중성 높은 초콜릿까지 폭 넓은 행사 상품 라인업을 준비했다.</div><div><br></div><div>핀란드 국민 초콜릿이라 불리는 칼파제르 밀크, 칼파제르 게이샤 헤이즐넛, 칼파제르 셀렉션(라즈베리 요거트/블루베리 트러플) 3종 상품(각 150g)을 판매하며 추가로 2+1 프로모션까지 진행한다.</div><div><br></div><div>1901년 스위스에서 처음 출시 됐으며 알프스 청정 지역의 우유를 더해 진한 부드러움이 가득하다고 알려진 밀카 초콜릿 알프스밀크, 오레오 2종 상품(각 100g)도 판매하며 2+1 프로모션이 추가 적용된다.</div><div><br></div><div>또, 1+1 대표상품으로 바삭하고 달달한 밀크 초코볼 허쉬후퍼스(340g)와 한입 크기 사이즈며 캐러

- 엔터테인먼트 주가 예측과 무관한 내용의 기사이므로 삭제한다

In [4]:
# 제거 후 확인
news = news.dropna(subset=['TITLE'])
print('TITLE 변수가 결측치인 행의 수:  ', sum(news['TITLE'].isna()),'개')

TITLE 변수가 결측치인 행의 수:   0 개


# 태그 제거 

- 원본 데이터의 손실을 막기 위해 복사본을 사용한다
- 뉴스 언론사별로 태그를 사용하는 방식이 다르므로 크게 두가지 방식으로 태그를 제거하고자 한다.
    - 방법1) p태그/태그가 없는 부분을 제외한 나머지 태그를 모두 제거하는 방식
        - 방법 1을 적용할 언론사 : 머니투데이, 뉴스핌, 한경뉴스 등
    - 방법2) a태그로 감싸져 있는 텍스트를 제거 & 모든 태그를 제거하는 방식
        - 방법 2를 적용할 언론사 : 로이터(AI뉴스), 인베스트조선, 인포스탁 등

In [5]:
news_copy = news.copy()

## 방법1(p 태그 + 미태그 제외하고 모두 제거)로 태그 제거

In [6]:
# 방법1로 태그 제거할 언론사 설정
comp1 = ['머니투데이', '뉴스핌', '한경뉴스', '서울경제', '파이낸셜 포인트', 'NSP', 'NSP통신', '파이낸셜뉴스']

# 태그 제거 
news_copy['PARSED CONTENT'] = ""

for i, row in news_copy.iterrows():
    if row['WRITER'] in comp1:
        html_content = row['CONTENT']
        soup = BeautifulSoup(html_content, 'html.parser')
        parsed_text = ""

        for content in soup.contents:
            if content.name == 'p':
                parsed_text += content.get_text() + "\n"
            elif isinstance(content, str):
                parsed_text += content + "\n"

        news_copy.at[i, 'PARSED CONTENT'] = parsed_text


In [7]:
news_copy['PARSED CONTENT'].iloc[0]  # 방법1로 태그 제거된 결과를 확인한기 위한 예시

'[머니투데이 변휘 기자, 김승한 기자] \nSK텔레콤·KT·LG유플러스 등 통신3사 CEO(최고경영자)들은 새해 한목소리로 \'혁신의 성과를 내보이겠다\'는 포부를 밝혔다. 그간 통신업에서 AI(인공지능)·디지털·미디어 등으로 신사업 확장을 추진해 왔던 만큼, 올해는 노력의 결과를 기술적 성취와 견조한 실적으로 현실화겠다는 의지다._x000D_\n_x000D_\n유영상 SK텔레콤 대표는 2일 SK텔레콤·SK브로드밴드 임직원에 보낸 이메일 신년사에서 "올해는 \'AI 컴퍼니\' 비전의 실천과 성과를 가시화하는 한 해가 돼야 한다"고 밝혔다._x000D_\n_x000D_\n유 대표는 "지난해 인플레이션과 탈(脫)세계화의 도래는 올해도 불확실한 경영환경 요인이다. \'넥스트 인터넷\' 후보인 대화형 AI, 메타버스, 웹3 등은 부침을 거듭하면서도 지속해서 발전할 것"이라 진단했다. 이어 "이런 상황에서 움츠려 있기보다는 에이닷(A.)의 성공적 안착으로 글로벌 AI 사업의 기반을 구축하고, 통신·미디어·엔터프라이즈 등 기존 사업은 AI로 재정의해야 한다"고 강조했다. _x000D_\n_x000D_\n특히 "궁극적으로 글로벌 빅테크 수준의 서비스와 기술 역량을 확보해 나가겠다"면서 "대한민국 통신과 ICT(정보통신기술) 역사를 개척해 온 자부심과 성공에 대한 확신을 가지고, 검은 토끼처럼 \'크게 도약해 크게 수확\'하는 기반을 만들자"고 당부했다. _x000D_\n_x000D_\n구현모 KT 대표는 "이익을 보장하는 성장, 미래에 인정받는 성장"을 내세웠다. 그는 이날 송파사옥에서 열린 신년식에서 "디지코(DIGICO·디지털플랫폼기업) 전략을 보다 확장하고, 다른 산업과의 연계 및 글로벌 진출로 3차원적 성장을 만들자"고 밝혔다._x000D_\n_x000D_\n 구 대표는 또 "디지털 시대의 리딩은 사업뿐만 아니라 사회적 책임까지 포함된다"며 "대한민국 발전에 기여하는 KT그룹이 돼야 한다"고 강조했다. 이어 "그 일환으로 디지털 시대 사회적 부작용에 대한 사명감을

## 방법2(a태그 텍스트 제거 + 태그 제거 )로 태그 제거 

In [8]:
# 방법2로 태그 제거할 언론사 설정
comp2 = ['로이터(AI뉴스)', '인베스트조선', '인포스탁', '연합뉴스', '뉴시스',
       'CEO 스코어데일리', '헤럴드경제', '뉴 프라임경제', '뉴스웨이', 'IR큐더스', '매일경제',
       '연합인포해외', '한경WOW해외', 'SCMP(AI뉴스)', 'CNBC(AI뉴스)', '아시아경제', '컨슈머타임스',
       'eDaily', '글로벌이코노믹', '데이터투자', '조선경제',  '머니투데이방송', '이투데이']

# 태그 제거 

for i, row in news_copy.iterrows():
    if row['WRITER'] in comp2:
        html_content = row['CONTENT']
        soup = BeautifulSoup(html_content, 'html.parser')

        for a_tag in soup.find_all('a'):
            a_tag.decompose()

        parsed_text = soup.get_text()
        news_copy.at[i, 'PARSED CONTENT'] = parsed_text

In [9]:
news_copy['PARSED CONTENT'].iloc[10000]  # 방법2로 태그 제거된 결과를 확인한기 위한 예시

"제목 : 에스엠(041510) 소폭 상승세 +3.88% 기업개요 음악 전문 종합엔터테인먼트 업체. 강타, BoA, 동방신기, 슈퍼주니어, 소녀시대, 샤이니, EXO, Red Velvet(레드벨벳), NCT, SuperM, 에스파(aespa) 등의 소속 가수를 보유. 주요사업으로 음반 제작, 퍼블리싱, 공연 기획 및 제작 등의 사업 등. 영상 콘텐츠 제작 및 매니지먼트, 여행업 등을 영위하는 코스닥 상장사 SM C&C, 키이스트, 에스엠라이프디자인그룹 등을 계열사로 보유.최대주주는 카카오 외(40.2%), 주요주주는 국민연금공단(8.96%),  Update : 2023.03.29        개인/외국인/기관 일별 순매매동향(수량기준, 전일까지 5거래일)일자종가(등락률)거래량개인외국인기관계기타04-21100,500 ( -2.43 % )269,352+19,097-25,523+5,861+56504-20103,000 ( -1.44 % )318,058-34,884+29,180+1,627+4,07704-19104,500 ( +1.75 % )314,819-9,035-23,026+30,115+1,94604-18102,700 ( -2.00 % )564,128+101,957-5,513-100,548+4,10404-17104,800 ( -0.66 % )273,654+7,789-44,941+34,200+2,952 공매도 정보(수량 기준, 전일까지 5거래일)일자공매도 거래량업틱룰 적용업틱룰 예외잔고수량매매비중04-219,2314,8174,414-3.43%04-2015,9875,21010,777-5.03%04-1911,6707,3704,300100,1523.71%04-1816,0703,13512,935114,7472.85%04-179,8791,4048,475133,8873.61%* 업틱룰예외는 해지거래, 차익거래 등 업틱룰의 적용이 면제된 거래를 의미.* 공매도잔고는 보고의무에 따라 당일 기준으로 2일전 내역까지 확인가능.* 매매비중 = 공매도 거래량 / 총 거래량* 출처 : KRX

- 파싱전 기사본문인 CONTENT열을 제외시키고 파싱후 기사본문인 PARSED CONTENT 변수를 포함하여 news_parsed 데이터프레임을 생성한다

In [10]:
# CONTENT열을 제외하고 news_parsed 데이터프레임에 저장
columns_to_keep = ['NEWS_YM', 'DATE_TIME', 'WRITER', 'SUBCATEGORY', 'TITLE', 
                   'IMPORTANCE', 'ITEM_NAME', 'TAG_LIST', 'PARSED CONTENT'] # 포함할 변수 목록
news_parsed = news_copy[columns_to_keep].copy()  # 데이터프레임 생성
news_parsed.head() # 데이터프레임 확인

,NEWS_YM,DATE_TIME,WRITER,SUBCATEGORY,TITLE,IMPORTANCE,ITEM_NAME,TAG_LIST,PARSED CONTENT
0,202301,20230102_135224,머니투데이,기술,"""'통신→디지털' 혁신을 성과로""…SKT·KT·LGU+ 신년 화두는",53.75,SK텔레콤|KT|LG유플러스,AI| 디지털| 통신| 미디어| 글로벌| 혁신| 인터넷| 플랫폼,"[머니투데이 변휘 기자, 김승한 기자] \nSK텔레콤·KT·LG유플러스 등 통신3사..."
1,202301,20230106_131618,머니투데이,경제,"구찌 티셔츠에 한글 로고, 가방엔 JEJU...""한국 독점 상품""",66.21,NaN,독점| 명품| 소비| 제주| 가방,[머니투데이 정인지 기자] \n글로벌 명품 브랜드들이 한국 소비자들 타깃으로 하는 ...
2,202301,20230111_120209,머니투데이,경제,"YG 트레저, 모바일 게임 '뿌까 퍼즐 어드벤처' 컬래버레이션",46.75,와이지엔터테인먼트,모바일| 게임| 캐릭터,[머니투데이 김건우 기자] \n_x000D_\n _x000D_\n와이지엔터테인먼트는...
3,202301,20230117_132642,머니투데이,사회,"현대오토에버, 1분기 신입·경력 채용 시작…""MZ가 면접관으로""",60.51,NaN,소프트웨어| 모빌리티| 문화| 교육| 자동차| 디지털,[머니투데이 정한결 기자] \n현대오토에버는 2023년 1분기 신입·경력 사원 채용...
4,202301,20230102_084347,eDaily,사회,BTS 자리 넘본다…유망주는 스트레이 키즈-KB,67.91,NaN,BTS| 판매량| NCT| 넥스트| 엔하이픈| 블랙핑크| 에스파| 아이브,- KB증권 “군 입대 BTS에 도전 이어질 것”_x000D_\n- “스트레이 키즈...


In [11]:
# news_parsed 기본정보 확인 
news_parsed.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16176 entries, 0 to 16176
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   NEWS_YM         16176 non-null  int64  
 1   DATE_TIME       16176 non-null  object 
 2   WRITER          16176 non-null  object 
 3   SUBCATEGORY     16176 non-null  object 
 4   TITLE           16176 non-null  object 
 5   IMPORTANCE      16176 non-null  float64
 6   ITEM_NAME       6889 non-null   object 
 7   TAG_LIST        11190 non-null  object 
 8   PARSED CONTENT  16176 non-null  object 
dtypes: float64(1), int64(1), object(7)
memory usage: 1.7+ MB


# 2차 키워드 필터링

- 태그 제거하기 전 K-POP 엔터테인먼트 관련 키워드로 1차 필터링을 진행했었음
    - News_Filtered_01.ipynb ~ News_Filtered_06.ipynb 파일에서 진행함
    - 엔터주가 예측과 무관한 기사를 제거함으로써 데이터 크기를 줄이기 위함(전처리 속도 높이기)
    - CONTENT(파싱 전 기사본문)변수에 엔터주가 예측에 필요한 키워드가 2번 이상 포함되어있지 않으면 제거하는 방식
- HTML의 태그가 제거된 뉴스 본문에 대하여 2차 필터링을 진행 
    - 'PARSED CONTENT'변수에 키워드를 3개 이상 포함하는 뉴스만 남도록 함

In [12]:
keywords = [
    '방시혁','하이브','에스엠','이수만','와이지','양현석','제이와이피','박진영',
    
    '뉴진스','방탄소년단','르세라핌',
    '세븐틴','투모로우바이투게더',
    '프로미스나인','나나','이현','미드낫','엔하이픈',
    '백호','강동호','황민현','예하나','성연','지코',
    '앤팀','히라테 유리나','보이넥스트도어',
    
    '블랙핑크','트레저','악뮤','악동뮤지션',
    '위너','지디','권지용',
    '젝스키스',
    
    '트와이스','엔믹스', '데이식스',
    'itzy','채령','류진','유나','예지'
    
    '스트레이키즈','투피엠','보이스토리',
    '야오천','니지유','엑스디너리 히어로즈',
    'boa',
    '에스파',
    '엑소',
    
    '샤이니','엔시티','엔시티 127','엔시티 드림',    
    '동방신기','슈퍼주니어','소녀시대','엑소'
    '레드벨벳','웨이션브이','슈퍼엠',
    '갓더비트','갓 더 비트','라이즈'

]

# 특정 단어 목록을 정규식 패턴으로 변환
keywords_pattern = '|'.join(keywords)

# 'PARSED CONTENT' 열에서 정규식 패턴을 검색하여 추출
filtered_df = news_parsed[news_parsed['PARSED CONTENT'].str.contains(keywords_pattern, flags=re.IGNORECASE, regex=True)]

# 특정 키워드가 3번 이상 포함된 행만 남기기
filtered_df = filtered_df[filtered_df['PARSED CONTENT'].apply(lambda text: sum(1 for keyword in keywords if keyword.lower() in text.lower()) >= 3)]


In [13]:
filtered_df.info() # 2차 필터링 결과 확인

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3577 entries, 4 to 16174
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   NEWS_YM         3577 non-null   int64  
 1   DATE_TIME       3577 non-null   object 
 2   WRITER          3577 non-null   object 
 3   SUBCATEGORY     3577 non-null   object 
 4   TITLE           3577 non-null   object 
 5   IMPORTANCE      3577 non-null   float64
 6   ITEM_NAME       1913 non-null   object 
 7   TAG_LIST        2685 non-null   object 
 8   PARSED CONTENT  3577 non-null   object 
dtypes: float64(1), int64(1), object(7)
memory usage: 279.5+ KB


# 언론사별로 전처리

- 태그 제거만으로도 어느정도 깔끔한 뉴스 기사 본문을 추출할 수 있음
- 하지만 본 조는 데이터의 퀄리티를 높이기 위해 기사본문에 포함된 불필요한 내용을 제거하고자 함 
    - ex)_x000D_, \n, \r
- 이러한 불필요한 부분은 언론사별로 규칙성을 가지고 있으므로, 정규표현식을 사용해 언론사별로 따로 처리하고자 함

## 로이터(AI뉴스) → 제거 
- 해외 언론사
- 엔터주 주가예측에 도움되는 뉴스가 없음을 확인함
- 해당 언론사의 기사 모두 제거

In [14]:
filtered_df = filtered_df[filtered_df['WRITER'] != '로이터(AI뉴스)']

## SCMP(AI뉴스)  → 제거
- 로이터(AI뉴스)와 마찬가지
- 해외 언론사
- 엔터주 주가예측에 도움되는 뉴스가 없음을 확인함
- 해당 언론사의 기사 모두 제거

In [15]:
filtered_df = filtered_df[filtered_df['WRITER'] != 'SCMP(AI뉴스)']

## 머니투데이

In [16]:
# 예시(전처리 전)
writer = '머니투데이'  # 언론사 설정 
filtered_df[filtered_df['WRITER']== writer]["PARSED CONTENT"].iloc[189]  # 확인


'[머니투데이 박수현 기자] \n키움증권이 15일 디어유에 대해 투자의견 \'매수\'를 유지하고 목표주가를 기존 6만8000원에서 6만5000원으로 하향 조정했다. 하반기로 갈수록 실적 개선 폭이 확대될 전망이라고 분석하면서도 타깃 주가수익비율(PER) 35배를 적용해 목표주가를 낮췄다._x000D_\n_x000D_\n이남수 키움증권 연구원은 "디어유의 1분기 매출액은 전년 동기 대비 40% 오른 164억원, 영업이익은 27% 오른 57억원으로 시장 기대치에 부합했다"며 "1분기 평균 구독 수는 전년 동기 대비 58% 상승한 205만명을 기록해 분기 기준 가장 큰 폭의 구독 수 변동이 발생했다"고 밝혔다._x000D_\n_x000D_\n이어 "에스엠 NCT, 에스파와 JYP의 스트레이키즈, 트와이스의 구독 수 증가가 지속됐다"며 "스타쉽 아이브, 몬스타엑스, IST 더보이즈 등도 1분기 런칭을 마쳤고 큐브 여자아이들, 비투비 등은 지난달 버블을 시작해 K-POP 주요 아티스트의 공격적 확장은 거의 마무리 단계"라고 했다._x000D_\n_x000D_\n이 연구원은 "에스엠 최대 주주 변경으로 카카오-에스엠-디어유 간의 시너지 강화를 기대한다"며 "1분기 현재 버블의 해외 비중은 77%로 문화 콘텐츠 흡수력 및 확장성이 가장 강한 글로벌 MZ세대에 호응을 얻고 있다. 카카오 플랫폼의 글로벌 확장 시 선두에 설 수 있는 가장 강력한 무기"라고 분석했다._x000D_\n_x000D_\n그러면서 "수량(Q) 확장 모멘텀이 국내 K-POP 아티스트에서 해외 현지로 확대되고 있다"며 "에스엠과 JYP 아티스트의 컴백과 월드투어가 계속되기에 하반기 성장성도 매우 높다. 3분기는 버블 재팬 런칭으로 해외 공략이 본격화될 것이다. 이후에는 미국, 중국 등으로 추가 확장을 추진할 것으로 전망한다"고 봤다._x000D_\n_x000D_\n\n박수현 기자 literature1028@mt.co.kr_x000D_\n_x000D_\n\ntextlinktopstart\n\n\n\n\n\n

In [17]:
# 불필요한 내용 삭제 1
filtered_df.loc[filtered_df['WRITER'] == writer, 'PARSED CONTENT'] = \
    filtered_df.loc[filtered_df['WRITER'] == writer, 'PARSED CONTENT'].str.replace("@mt.co.kr_x000D_\n_x000D_\n\ntextlinktopstart\n\n\n\n\n\n\n\n\ntextlinktopend\npowerIntstart\n", "", regex=False)



In [18]:
# 불필요한 내용 삭제 2
filtered_df.loc[filtered_df['WRITER'] == writer, 'PARSED CONTENT'] = \
    filtered_df.loc[filtered_df['WRITER'] == writer, 'PARSED CONTENT'].str.replace("※ 우울감 등 말하기 어려운 고민이 있거나 주변에 이런 어려움을 겪는 가족·지인이 있을 경우 자살예방 상담전화 ☎1393, 정신건강 상담전화 ☎1577-0199, 희망의 전화 ☎129, 생명의 전화 ☎1588-9191, 청소년 전화 ☎1388, 청소년 모바일 상담 '다 들어줄 개' 앱, 카카오톡 등에서 24시간 전문가의 상담을 받을 수 있습니다.", "", regex=False)


In [19]:
# 불필요한 내용 삭제 3
# 기자] 이전 내용 삭제
pattern = r'.*기자\]'
filtered_df.loc[filtered_df['WRITER'] == writer, 'PARSED CONTENT'] = \
    filtered_df.loc[filtered_df['WRITER'] == writer, 'PARSED CONTENT'].apply(lambda text: re.sub(pattern, '', text))


In [20]:
# 예시(전처리 후)
filtered_df[filtered_df['WRITER']== writer]["PARSED CONTENT"].iloc[189]

' \n키움증권이 15일 디어유에 대해 투자의견 \'매수\'를 유지하고 목표주가를 기존 6만8000원에서 6만5000원으로 하향 조정했다. 하반기로 갈수록 실적 개선 폭이 확대될 전망이라고 분석하면서도 타깃 주가수익비율(PER) 35배를 적용해 목표주가를 낮췄다._x000D_\n_x000D_\n이남수 키움증권 연구원은 "디어유의 1분기 매출액은 전년 동기 대비 40% 오른 164억원, 영업이익은 27% 오른 57억원으로 시장 기대치에 부합했다"며 "1분기 평균 구독 수는 전년 동기 대비 58% 상승한 205만명을 기록해 분기 기준 가장 큰 폭의 구독 수 변동이 발생했다"고 밝혔다._x000D_\n_x000D_\n이어 "에스엠 NCT, 에스파와 JYP의 스트레이키즈, 트와이스의 구독 수 증가가 지속됐다"며 "스타쉽 아이브, 몬스타엑스, IST 더보이즈 등도 1분기 런칭을 마쳤고 큐브 여자아이들, 비투비 등은 지난달 버블을 시작해 K-POP 주요 아티스트의 공격적 확장은 거의 마무리 단계"라고 했다._x000D_\n_x000D_\n이 연구원은 "에스엠 최대 주주 변경으로 카카오-에스엠-디어유 간의 시너지 강화를 기대한다"며 "1분기 현재 버블의 해외 비중은 77%로 문화 콘텐츠 흡수력 및 확장성이 가장 강한 글로벌 MZ세대에 호응을 얻고 있다. 카카오 플랫폼의 글로벌 확장 시 선두에 설 수 있는 가장 강력한 무기"라고 분석했다._x000D_\n_x000D_\n그러면서 "수량(Q) 확장 모멘텀이 국내 K-POP 아티스트에서 해외 현지로 확대되고 있다"며 "에스엠과 JYP 아티스트의 컴백과 월드투어가 계속되기에 하반기 성장성도 매우 높다. 3분기는 버블 재팬 런칭으로 해외 공략이 본격화될 것이다. 이후에는 미국, 중국 등으로 추가 확장을 추진할 것으로 전망한다"고 봤다._x000D_\n_x000D_\n\n박수현 기자 literature1028'

## 인베스트 조선

In [21]:
# 예시(전처리 전)
writer = '인베스트조선'  # 언론사 설정 
filtered_df[filtered_df['WRITER']== writer]["PARSED CONTENT"].iloc[9]  # 확인

'하이브의 퇴각으로 SM엔터테인먼트(이하 SM) 인수전이 정리된 가운데 카카오와 하이브의 \'동맹\'이 어느 수준으로 이어질지 주목된다. SM의 주인이 누가 되느냐 보다는 \'팬쉽 플랫폼\'의 주도권이 어디로 가느냐가 이번 경영권 분쟁의 핵심이었던 만큼 양쪽의 실익 계산이 치열하게 이어질 전망이다. 하이브와 카카오 사이에서 입지가 \'붕 뜬\' 이수만 전 SM 총괄 프로듀서가 개인회사 지분 처분, 주주제안 철회 등 문제를 어떻게 정리할지도 관심사다.15일 열린 관훈포럼에서 방시혁 하이브 의장은 "SM 인수전을 승패로 보지 않는다”며 “우리 미래에 가장 중요한 축인 플랫폼에 관해 카카오와 합의를 끌어내 개인적으로는 아주 만족하고 있다"고 말했다. 구체적 협력 방안에 관한 질문에는 "아직은 말씀드릴 수 있는 상황이 아니고 이른 시일 안에 실질적 협력이 되도록 준비하고 있다"고 답했다.양측의 ‘플랫폼 협업’ 방안은 팬쉽 플랫폼 문제가 핵심일 것으로 관측된다. 애초부터 ‘SM 회사’ 자체는 어느 기업에 인수되어도 크게 달라질 게 없고, 팬쉽 플랫폼을 누가 이끌게 되느냐는 것이 핵심이라는 평가가 있었다.하이브가 SM을 인수했다고 가정하면 가장 크게 달라질 것은 팬쉽 플랫폼 구도다. 현재 국내 팬쉽 플랫폼은 하이브의 위버스, SM의 디어유(버블) 양강 체제다. NC소프트의 ‘유니버스’는 올해 초 디어유가 IP 계약권 일체를 인수하며 문을 닫았다.엄밀히는 위버스와 버블은 서비스가 완전히 겹치지 않는다. 위버스는 아티스트별 ‘팬덤 활동’에 초점이 맞춰진,반면 버블은 아티스트와의 ‘프라이빗 메시지’ 서비스가 핵심이다. 위버스와 버블에 동시에 입점한 아티스트들도 다수라는 점을 고려하면 이 둘을 완전한 경쟁구도로 보긴 어렵다.다만 하이브가 최근 위버스에서 유료 아티스트 메시지 서비스를 선보일 움직임을 보이며 경쟁 압박이 커졌다. 하이브가 SM을 인수하면 위버스에 SM 아티스트들이 버블에 하이브 아티스트들이 입점하는 방식의 협력이 예상됐다. 상대적으로 위버스에 역량을 집중할 것이란

In [22]:
# 불필요한 내용 삭제 1
filtered_df.loc[filtered_df['WRITER'] == writer, 'PARSED CONTENT'] = \
    filtered_df.loc[filtered_df['WRITER'] == writer, 'PARSED CONTENT'].str.replace("인베스트조선", "", regex=False)

In [23]:
# 예시(전처리 후)
filtered_df[filtered_df['WRITER']== writer]["PARSED CONTENT"].iloc[9]

'하이브의 퇴각으로 SM엔터테인먼트(이하 SM) 인수전이 정리된 가운데 카카오와 하이브의 \'동맹\'이 어느 수준으로 이어질지 주목된다. SM의 주인이 누가 되느냐 보다는 \'팬쉽 플랫폼\'의 주도권이 어디로 가느냐가 이번 경영권 분쟁의 핵심이었던 만큼 양쪽의 실익 계산이 치열하게 이어질 전망이다. 하이브와 카카오 사이에서 입지가 \'붕 뜬\' 이수만 전 SM 총괄 프로듀서가 개인회사 지분 처분, 주주제안 철회 등 문제를 어떻게 정리할지도 관심사다.15일 열린 관훈포럼에서 방시혁 하이브 의장은 "SM 인수전을 승패로 보지 않는다”며 “우리 미래에 가장 중요한 축인 플랫폼에 관해 카카오와 합의를 끌어내 개인적으로는 아주 만족하고 있다"고 말했다. 구체적 협력 방안에 관한 질문에는 "아직은 말씀드릴 수 있는 상황이 아니고 이른 시일 안에 실질적 협력이 되도록 준비하고 있다"고 답했다.양측의 ‘플랫폼 협업’ 방안은 팬쉽 플랫폼 문제가 핵심일 것으로 관측된다. 애초부터 ‘SM 회사’ 자체는 어느 기업에 인수되어도 크게 달라질 게 없고, 팬쉽 플랫폼을 누가 이끌게 되느냐는 것이 핵심이라는 평가가 있었다.하이브가 SM을 인수했다고 가정하면 가장 크게 달라질 것은 팬쉽 플랫폼 구도다. 현재 국내 팬쉽 플랫폼은 하이브의 위버스, SM의 디어유(버블) 양강 체제다. NC소프트의 ‘유니버스’는 올해 초 디어유가 IP 계약권 일체를 인수하며 문을 닫았다.엄밀히는 위버스와 버블은 서비스가 완전히 겹치지 않는다. 위버스는 아티스트별 ‘팬덤 활동’에 초점이 맞춰진,반면 버블은 아티스트와의 ‘프라이빗 메시지’ 서비스가 핵심이다. 위버스와 버블에 동시에 입점한 아티스트들도 다수라는 점을 고려하면 이 둘을 완전한 경쟁구도로 보긴 어렵다.다만 하이브가 최근 위버스에서 유료 아티스트 메시지 서비스를 선보일 움직임을 보이며 경쟁 압박이 커졌다. 하이브가 SM을 인수하면 위버스에 SM 아티스트들이 버블에 하이브 아티스트들이 입점하는 방식의 협력이 예상됐다. 상대적으로 위버스에 역량을 집중할 것이란

## 인포스탁

In [24]:
# 예시(전처리 전)
writer = '인포스탁'  # 언론사 설정 
filtered_df[filtered_df['WRITER']== writer]["PARSED CONTENT"].iloc[15]  # 확인

'제목 : 팬엔터테인먼트(068050) 소폭 상승세 +3.07% 기업개요 드라마, 교양/예능프로그램 등의 방송영상물제작 및 드라마OST 등의 음반제작을 주요사업으로 영위하는 업체. 주요작품으로 해를 품은 달, 최고의연인, 동백꽃 필 무렵, 두 번은 없다, 청춘 기록 등이 있음. 신규사업으로 영화제작을 진행하고 있으며, 시나리오 개발 및 웹툰, 소설 등 원작확보를 통해 자제제작 및 공동제작을 준비중.최대주주는 박영석(35.9%)  Update : 2022.12.09        개인/외국인/기관 일별 순매매동향(수량기준, 전일까지 5거래일)일자종가(등락률)거래량개인외국인기관계기타01-124,710 ( -1.46 % )248,326-6,973+3,805+1,803+1,36501-114,780 ( +1.59 % )376,418+47,110+38,006-88,449+3,33301-104,705 ( -5.52 % )903,298+141,923-153,348+19,741-8,31601-094,980 ( -0.99 % )464,110-110,652+64,318+46,334001-065,030 ( -2.52 % )451,369+124,496-54,368-65,848-4,280 공매도 정보(수량 기준, 전일까지 5거래일)일자공매도 거래량업틱룰 적용업틱룰 예외잔고수량매매비중01-12000-0.00%01-1134034-0.01%01-103003000.00%01-09142014200.03%01-06153015300.03%* 업틱룰예외는 해지거래, 차익거래 등 업틱룰의 적용이 면제된 거래를 의미.* 공매도잔고는 보고의무에 따라 당일 기준으로 2일전 내역까지 확인가능.* 매매비중 = 공매도 거래량 / 총 거래량* 출처 : KRX 정보데이터시스템 종목히스토리 ▶ 종목이슈 ☞ 2022-12-14 급등 - 글로벌 OTT 한국 드라마 투자 확대 기대감 등에 영상콘텐츠 테마 상승 속 급등 ☞ 2022-12-09 급등 - 국내 드라마 中 OTT 방영 소식 속 한한령 해제 기대감 등에 영상콘텐츠/

In [25]:
# 불필요한 내용 삭제 1
# 공매도 정보(수량 기준, 이후의 모든 내용 삭제
pattern = r'공매도 정보\(수량 기준.*$'
filtered_df.loc[filtered_df['WRITER'] == writer, 'PARSED CONTENT'] = \
    filtered_df.loc[filtered_df['WRITER'] == writer, 'PARSED CONTENT'].apply(lambda text: re.sub(pattern, '', text))


In [26]:
# 불필요한 내용 삭제 2
# 개인/외국인/기관 일별 순매매동향 이후의 모든 내용 삭제

pattern = r'개인/외국인/기관 일별 순매매동향.*$'
filtered_df.loc[filtered_df['WRITER'] == writer, 'PARSED CONTENT'] = \
    filtered_df.loc[filtered_df['WRITER'] == writer, 'PARSED CONTENT'].apply(lambda text: re.sub(pattern, '', text))


In [27]:
# 불필요한 내용 삭제 3
# Update : 이후의 모든 내용 삭제
pattern = r'Update :.*$'
filtered_df.loc[filtered_df['WRITER'] == writer, 'PARSED CONTENT'] = \
    filtered_df.loc[filtered_df['WRITER'] == writer, 'PARSED CONTENT'].apply(lambda text: re.sub(pattern, '', text))

In [28]:
# 예시(전처리 후)
filtered_df[filtered_df['WRITER']== writer]["PARSED CONTENT"].iloc[15]  

'제목 : 팬엔터테인먼트(068050) 소폭 상승세 +3.07% 기업개요 드라마, 교양/예능프로그램 등의 방송영상물제작 및 드라마OST 등의 음반제작을 주요사업으로 영위하는 업체. 주요작품으로 해를 품은 달, 최고의연인, 동백꽃 필 무렵, 두 번은 없다, 청춘 기록 등이 있음. 신규사업으로 영화제작을 진행하고 있으며, 시나리오 개발 및 웹툰, 소설 등 원작확보를 통해 자제제작 및 공동제작을 준비중.최대주주는 박영석(35.9%)  '

## 연합뉴스

In [29]:
# 예시(전처리 전)
writer = '연합뉴스'  # 언론사 설정 
filtered_df[filtered_df['WRITER']== writer]["PARSED CONTENT"].iloc[100]  # 확인

'하이브·카카오, SM 쩐의 전쟁…"끝날 때까지 끝난 게 아니다"(종합)\n얼라인 공개매수 12만원 책정에 반발…"지분 경쟁 속 SM 주가에 긍정적"_x000D_\n_x000D_\n    (서울=연합인포맥스) 장순환 정지서 한상민 기자 = SM엔터테인먼트(이하 에스엠) 경영권 _x000D_\n분쟁에 하이브가 참전하면서 에스엠과 하이브의 주가가 동반 급등하고 있다._x000D_\n    하이브는 에스엠 창업자인 이수만 대주주의 지분을 인수하며 최대 주주에 오른 후 소액주_x000D_\n주들의 공개 매수 의지도 밝힌 만큼 카카오와 본격적인 지분 경쟁으로 향후 주가 상승세는 이_x000D_\n어질 전망이다._x000D_\n    10일 연합인포맥스 종목 현재가(화면번호3111)에 따르면 에스엠은 오전 11시 15분 현재 기_x000D_\n준 전 거래일보다 16.35% 오른 11만4천600원에 거래되고 있다._x000D_\n    유가증권시장에서 하이브도 8.17% 오른 21만4천500원에, 카카오는 3.81% 내린 6만8천300원_x000D_\n에 거래 중이다._x000D_\n    하이브와 카카오는 에스엠 인수로 업계 1위를 놓고 치열한 신경전을 벌이고 있는 만큼 향_x000D_\n후 치열한 지분 확보 전쟁 가능성이 있다._x000D_\n    하이브는 현재 업계 1위라는 타이틀과 명성은 엔터 업계에서 강한 영향력을 유지하고 있다_x000D_\n._x000D_\n    다만, 카카오엔터테인먼트 상장 시 단숨에 시총 1위를 넘겨줄 가능성이 있어 카카오엔터테_x000D_\n인먼트를 견제할 수밖에 없는 상황이다._x000D_\n    이에 하이브는 이날 이수만 전 총괄 프로듀서가 보유한 지분 14.8%를 4천228억원에 인수하_x000D_\n는 계약을 체결해 에스엠의 단독 1대 주주에 올랐다._x000D_\n    증권가에서는 하이브가 에스엠 지분 인수를 완료하면 K팝 1군 IP(지식재산권)를 확보한 최_x000D_\n대 사업자의 지위가 더욱 공고해질 것이라고 보고 있

In [30]:
# 불필요한 내용 삭제 1
# ┌ 이후 내용 모두 삭제
pattern = r'┌.*$'
filtered_df.loc[filtered_df['WRITER'] == writer, 'PARSED CONTENT'] = \
    filtered_df.loc[filtered_df['WRITER'] == writer, 'PARSED CONTENT'].apply(lambda text: re.sub(pattern, '', text))

In [31]:
# 불필요한 내용 삭제 1
# (끝) 이후 내용 모두 삭제

pattern = r'\(끝\).*'
filtered_df.loc[filtered_df['WRITER'] == writer, 'PARSED CONTENT'] = \
    filtered_df.loc[filtered_df['WRITER'] == writer, 'PARSED CONTENT'].apply(lambda text: re.sub(pattern, '', text))


In [32]:
# 불필요한 내용 삭제 2
# 기자 = 이전내용 모두 삭제
pattern = r'.*기자 ='
filtered_df.loc[filtered_df['WRITER'] == writer, 'PARSED CONTENT'] = \
    filtered_df.loc[filtered_df['WRITER'] == writer, 'PARSED CONTENT'].apply(lambda text: re.sub(pattern, '', str(text), flags=re.DOTALL | re.IGNORECASE))


In [33]:
# 불필요한 내용 삭제 3
# 불필요한 내용 삭제
filtered_df.loc[filtered_df['WRITER'] == writer, 'PARSED CONTENT'] = \
    filtered_df.loc[filtered_df['WRITER'] == writer, 'PARSED CONTENT'].str.replace("<저작권자(c) 연합뉴스, 무단 전재-재배포 금지>", "", regex=False)



In [34]:
# 예시(전처리 후)
filtered_df[filtered_df['WRITER']== writer]["PARSED CONTENT"].iloc[70]

" 그룹 투모로우바이투게더(TXT)가 다섯 번째 미니음반 '이_x000D_\n름의 장 : 템프테이션(TEMPTATION)'으로 미국 빌보드 메인 앨범 차트 '빌보드 200' 1위에 올랐_x000D_\n다._x000D_\n    빌보드는 5일(현지시간) 차트 예고 기사에서 투모로우바이투게더가 '빌보드 200' 정상에 _x000D_\n올랐다고 전했다. 이는 지난 2019년 3월 이들이 데뷔한 지 3년 11개월 만이다._x000D_\n    '빌보드 200' 1위를 밟은 한국 가수는 방탄소년단(BTS·2018년), 슈퍼엠(2019년), 스트레_x000D_\n이 키즈(2022년), 블랙핑크(2022년)에 이어 투모로우바이투게더가 다섯 번째다. _x000D_\n    방탄소년단과 투모로우바이투게더의 소속사 빅히트뮤직(하이브)은 빌보드 메인 차트에 두 _x000D_\n팀 이상의 가수를 1위에 올려놓은 유일한 국내 가요 기획사라는 기록도 세웠다._x000D_\n    tsl@yna.co.kr_x000D_\n\n\n_x000D_\n                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                

## 뉴시스

In [35]:
# 예시(전처리 전)
writer = '뉴시스'  # 언론사 설정 
filtered_df[filtered_df['WRITER']== writer]["PARSED CONTENT"].iloc[0]  # 확인

'[서울=뉴시스]이재훈 기자 = 에스엠(SM)엔터테인먼트가 임시 사외이사후보 추천위원회(이하 \'임시사추위\')를 구성했다고 25일 밝혔다. \n\n사내이사 위원으로 이성수 SM 대표이사가, 외부위원 중 이사회 추천으로 조명현 고려대학교 경영학과 교수가, 외부위원 중 얼라인 측 추천위원으로 이남우 연세대학교 국제학대학원 객원교수가 임명됐다. \n\n조 교수는 한국기업지배구조원 원장 및 한국스튜어드십코드 제정위원장을 지낸 기업거버넌스 전문가다. 대한항공, 현대글로비스 등 기업의 사외이사로 활동하고 있다. SM이사회는 글로벌 자문기관을 통해 조 교수를 추천 받아 임시 사추위 위원으로 선임했다. \n\n이 객원교수는 뱅크오브아메리카 메릴린치 한국공동대표, 노무라증권 아시아 고객관리총괄 등 국제금융업계에서 일한 금융 및 투자전문가다. 최근에는 한국투자신탁운용, 에이비엘바이오, 한솔홀딩스 등의 기업에서 사외이사로 활약했다. \n\n이번 임명은 SM이 지난 22일 이사회에서 얼라인파트너스자산운용 주식회사(이하 얼라인)와 함께 발표한 주주가치 제고 및 거버넌스 개선을 위한 12개 합의사항 중 하나다. 당일 임시사추위를 비롯 주주환원정책과 관련한 3개 의안을 이사 4인 중 사내이사 3인의 찬성, 사외이사 기권으로 결의했다. \n\n또한, SM 이사회는 앞으로 3년간 별도 당기순이익의 20% 이상을 주주에게 환원한다는 주주환원정책도 결의했다.\nSM은 "이번 주주환원정책의 공시 및 이사회 의결을 통해 다가올 금년 3월 주총에서부터 현금배당 또는 자사주 소각을 통해 주주들에게 회사의 이익을 환원할 계획"이라고 예고했다. \n\n얼라인과  합의 사항 중 가장 중요한 부분 중 하나는 멀티 프로듀싱 체제다. 그간 SM은 약 27년 간 이수만 총괄 프로듀서의 진두지휘(陣頭指揮) 아래 SM컬처유니버스(SMCU·SM Culture Universe)의 세계관을 쌓아왔다. SMCU는 다양한 영웅을 내세운 디즈니의 마블 스튜디오처럼, SM에 속한 각 그룹이 이른바 \'광야\'(KWANGYA)라

In [36]:
# 불필요한 내용 삭제 1
# 기자 =  이전 내용 삭제
pattern = r'.*기자\s='
filtered_df.loc[filtered_df['WRITER'] == writer, 'PARSED CONTENT'] = \
    filtered_df.loc[filtered_df['WRITER'] == writer, 'PARSED CONTENT'].apply(lambda text: re.sub(pattern, '', text))

In [37]:
filtered_df[filtered_df['WRITER']== writer]["PARSED CONTENT"].iloc[0]  # 확인

' 에스엠(SM)엔터테인먼트가 임시 사외이사후보 추천위원회(이하 \'임시사추위\')를 구성했다고 25일 밝혔다. \n\n사내이사 위원으로 이성수 SM 대표이사가, 외부위원 중 이사회 추천으로 조명현 고려대학교 경영학과 교수가, 외부위원 중 얼라인 측 추천위원으로 이남우 연세대학교 국제학대학원 객원교수가 임명됐다. \n\n조 교수는 한국기업지배구조원 원장 및 한국스튜어드십코드 제정위원장을 지낸 기업거버넌스 전문가다. 대한항공, 현대글로비스 등 기업의 사외이사로 활동하고 있다. SM이사회는 글로벌 자문기관을 통해 조 교수를 추천 받아 임시 사추위 위원으로 선임했다. \n\n이 객원교수는 뱅크오브아메리카 메릴린치 한국공동대표, 노무라증권 아시아 고객관리총괄 등 국제금융업계에서 일한 금융 및 투자전문가다. 최근에는 한국투자신탁운용, 에이비엘바이오, 한솔홀딩스 등의 기업에서 사외이사로 활약했다. \n\n이번 임명은 SM이 지난 22일 이사회에서 얼라인파트너스자산운용 주식회사(이하 얼라인)와 함께 발표한 주주가치 제고 및 거버넌스 개선을 위한 12개 합의사항 중 하나다. 당일 임시사추위를 비롯 주주환원정책과 관련한 3개 의안을 이사 4인 중 사내이사 3인의 찬성, 사외이사 기권으로 결의했다. \n\n또한, SM 이사회는 앞으로 3년간 별도 당기순이익의 20% 이상을 주주에게 환원한다는 주주환원정책도 결의했다.\nSM은 "이번 주주환원정책의 공시 및 이사회 의결을 통해 다가올 금년 3월 주총에서부터 현금배당 또는 자사주 소각을 통해 주주들에게 회사의 이익을 환원할 계획"이라고 예고했다. \n\n얼라인과  합의 사항 중 가장 중요한 부분 중 하나는 멀티 프로듀싱 체제다. 그간 SM은 약 27년 간 이수만 총괄 프로듀서의 진두지휘(陣頭指揮) 아래 SM컬처유니버스(SMCU·SM Culture Universe)의 세계관을 쌓아왔다. SMCU는 다양한 영웅을 내세운 디즈니의 마블 스튜디오처럼, SM에 속한 각 그룹이 이른바 \'광야\'(KWANGYA)라는 활동 공간에서 각자 쌓아온

In [38]:
# 불필요한 내용 삭제 2
# ◎이후 내용 삭제

pattern = r'◎.*$'
filtered_df.loc[filtered_df['WRITER'] == writer, 'PARSED CONTENT'] = \
    filtered_df.loc[filtered_df['WRITER'] == writer, 'PARSED CONTENT'].apply(lambda text: re.sub(pattern, '', text) if isinstance(text, str) else text)


In [39]:
# 예시(전처리 후)
filtered_df[filtered_df['WRITER']== writer]["PARSED CONTENT"].iloc[0]

' 에스엠(SM)엔터테인먼트가 임시 사외이사후보 추천위원회(이하 \'임시사추위\')를 구성했다고 25일 밝혔다. \n\n사내이사 위원으로 이성수 SM 대표이사가, 외부위원 중 이사회 추천으로 조명현 고려대학교 경영학과 교수가, 외부위원 중 얼라인 측 추천위원으로 이남우 연세대학교 국제학대학원 객원교수가 임명됐다. \n\n조 교수는 한국기업지배구조원 원장 및 한국스튜어드십코드 제정위원장을 지낸 기업거버넌스 전문가다. 대한항공, 현대글로비스 등 기업의 사외이사로 활동하고 있다. SM이사회는 글로벌 자문기관을 통해 조 교수를 추천 받아 임시 사추위 위원으로 선임했다. \n\n이 객원교수는 뱅크오브아메리카 메릴린치 한국공동대표, 노무라증권 아시아 고객관리총괄 등 국제금융업계에서 일한 금융 및 투자전문가다. 최근에는 한국투자신탁운용, 에이비엘바이오, 한솔홀딩스 등의 기업에서 사외이사로 활약했다. \n\n이번 임명은 SM이 지난 22일 이사회에서 얼라인파트너스자산운용 주식회사(이하 얼라인)와 함께 발표한 주주가치 제고 및 거버넌스 개선을 위한 12개 합의사항 중 하나다. 당일 임시사추위를 비롯 주주환원정책과 관련한 3개 의안을 이사 4인 중 사내이사 3인의 찬성, 사외이사 기권으로 결의했다. \n\n또한, SM 이사회는 앞으로 3년간 별도 당기순이익의 20% 이상을 주주에게 환원한다는 주주환원정책도 결의했다.\nSM은 "이번 주주환원정책의 공시 및 이사회 의결을 통해 다가올 금년 3월 주총에서부터 현금배당 또는 자사주 소각을 통해 주주들에게 회사의 이익을 환원할 계획"이라고 예고했다. \n\n얼라인과  합의 사항 중 가장 중요한 부분 중 하나는 멀티 프로듀싱 체제다. 그간 SM은 약 27년 간 이수만 총괄 프로듀서의 진두지휘(陣頭指揮) 아래 SM컬처유니버스(SMCU·SM Culture Universe)의 세계관을 쌓아왔다. SMCU는 다양한 영웅을 내세운 디즈니의 마블 스튜디오처럼, SM에 속한 각 그룹이 이른바 \'광야\'(KWANGYA)라는 활동 공간에서 각자 쌓아온

## CEO 스코어데일리

In [40]:
# 예시(전처리 전)
writer = 'CEO 스코어데일리'  # 언론사 설정 
filtered_df[filtered_df['WRITER']== writer]["PARSED CONTENT"].iloc[0]  # 확인

'카카오의 올해 ‘비욘드 코리아’ 전략이 연초부터 삐걱거리고 있다. 카카오는 SM엔터테인먼트 인수전에 지원군으로 나서 이 회사 소속 아티스트들의 IP(지식재산)를 확보할 방침이었지만, 그룹 방탄소년단(BTS) 소속사인 하이브가 물량공세에 나서면서 비상등이 켜진 것이다. 13일 관련업계에 따르면 카카오는 지난 7일 SM엔터테인먼트의 지분 9.05%를 2171억원에 취득하기로 했다고 밝혔다. SM이 제3자 배정 유상증자 형태로 발행하는 123만주 규모의 신주를 인수하고, 전환사채 인수를 통해 보통주 전환 기준 114만주를 확보하는 방식이다. 취득 예정일자는 내달 6일이다. 이번 투자가 성사되면 카카오는 SM의 2대 주주로 올라, SM이 확보한 콘텐츠를 활용할 수 있게 된다. 카카오는 이번 투자를 계기로 카카오엔터테인먼트와 SM이 각사의 해외 파트너 등 네트워크를 활용해 글로벌 매니지먼트 사업을 함께 추진하고, 글로벌 오디션을 통해 K-팝 아티스트를 공동 기획하는 등 IP 경쟁력 강화를 위한 협력에도 본격적으로 나설 계획이었다. 배재현 카카오 수석부사장은 지난 10일 진행된 컨퍼런스 콜에서 “웹툰, 웹소설 및 캐릭터 굿즈 사업을 결합해 SM의 IP 사업 활용도를 지금보다 높이겠다”며 “팬플랫폼 사업과 관련해 카카오 아티스트 협력, IT 역량 기반으로 팬플랫폼을 확장할 수 있을 것”이라고 설명한 바 있다. 하지만 카카오의 이같은 계획은 하이브가 SM 창업자인 이수만 전 총괄 프로듀서의 ‘백기사’로 부상하면서 물거품 될 위기를 맞고 있다. 하이브는 지난 10일 이수만 전 총괄 프로듀서가 보유한 지분 14.8%를 4228억원에 인수한다고 전격적으로 발표했다. 취득 예정 일자는 다음달 6일이다.  SM전현직 경영진을 대신해 하이브와카카오가 대리전을 벌이고 있는 형국이다. SM은 지난 9월 말까지만 해도 이수만(18.46%), 국민연금(8.96%), KB자산운용(5.12%), 컴투스(4.2%)등이 지분을 보유하고 있었지만, 하이브가 이수만 전 총괄의 지분 14.8%를 인수하

In [41]:
# 불필요한 내용 삭제 1
# [CEO스코어데일리 이후내용 삭제 
pattern = r'\[CEO스코어데일리[^\]]*\]'
filtered_df.loc[filtered_df['WRITER'] == writer, 'PARSED CONTENT'] = \
    filtered_df.loc[filtered_df['WRITER'] == writer, 'PARSED CONTENT'].apply(lambda text: re.sub(pattern, '', text))

In [42]:
# 예시(전처리 후)
filtered_df[filtered_df['WRITER']== writer]["PARSED CONTENT"].iloc[0]

'카카오의 올해 ‘비욘드 코리아’ 전략이 연초부터 삐걱거리고 있다. 카카오는 SM엔터테인먼트 인수전에 지원군으로 나서 이 회사 소속 아티스트들의 IP(지식재산)를 확보할 방침이었지만, 그룹 방탄소년단(BTS) 소속사인 하이브가 물량공세에 나서면서 비상등이 켜진 것이다. 13일 관련업계에 따르면 카카오는 지난 7일 SM엔터테인먼트의 지분 9.05%를 2171억원에 취득하기로 했다고 밝혔다. SM이 제3자 배정 유상증자 형태로 발행하는 123만주 규모의 신주를 인수하고, 전환사채 인수를 통해 보통주 전환 기준 114만주를 확보하는 방식이다. 취득 예정일자는 내달 6일이다. 이번 투자가 성사되면 카카오는 SM의 2대 주주로 올라, SM이 확보한 콘텐츠를 활용할 수 있게 된다. 카카오는 이번 투자를 계기로 카카오엔터테인먼트와 SM이 각사의 해외 파트너 등 네트워크를 활용해 글로벌 매니지먼트 사업을 함께 추진하고, 글로벌 오디션을 통해 K-팝 아티스트를 공동 기획하는 등 IP 경쟁력 강화를 위한 협력에도 본격적으로 나설 계획이었다. 배재현 카카오 수석부사장은 지난 10일 진행된 컨퍼런스 콜에서 “웹툰, 웹소설 및 캐릭터 굿즈 사업을 결합해 SM의 IP 사업 활용도를 지금보다 높이겠다”며 “팬플랫폼 사업과 관련해 카카오 아티스트 협력, IT 역량 기반으로 팬플랫폼을 확장할 수 있을 것”이라고 설명한 바 있다. 하지만 카카오의 이같은 계획은 하이브가 SM 창업자인 이수만 전 총괄 프로듀서의 ‘백기사’로 부상하면서 물거품 될 위기를 맞고 있다. 하이브는 지난 10일 이수만 전 총괄 프로듀서가 보유한 지분 14.8%를 4228억원에 인수한다고 전격적으로 발표했다. 취득 예정 일자는 다음달 6일이다.  SM전현직 경영진을 대신해 하이브와카카오가 대리전을 벌이고 있는 형국이다. SM은 지난 9월 말까지만 해도 이수만(18.46%), 국민연금(8.96%), KB자산운용(5.12%), 컴투스(4.2%)등이 지분을 보유하고 있었지만, 하이브가 이수만 전 총괄의 지분 14.8%를 인수하

## 헤럴드경제

In [43]:
# 예시(전처리 전)
writer = '헤럴드경제'  # 언론사 설정 
filtered_df[filtered_df['WRITER']== writer]["PARSED CONTENT"].iloc[1]  # 확인

'새 걸그룹 ‘베이비몬스터’ 실체는\n지드래곤 등 아티스트 해외공연\n올 계약 만료 블랙핑크와 재계약\n더블랙레이블과 협력도 과제양현석 총괄 프로듀서 복귀와 새 걸그룹 ‘베이비몬스터’의 등판으로 새해 첫날 10% 가까운 급등세를 보인 와이지엔터테인먼트 주가가 다시 횡보세로 돌아섰다.6일 오전엔 4% 가까운 급락세를 보였다. 아직 정보가 부족한 걸그룹에 대한 기대감만으로 상승세를 이어가기는 역부족인 것으로 풀이된다. 전문가는 베이비몬스터의 성공과 함께 ▷지드래곤을 비롯한 남은 아티스트의 해외공연 ▷블랙핑크와 재계약 ▷관계사인 더블랙레이블과 지속협력 등이 이어져야 주가가 지속적인 탄력을 받을 수 있을 것으로 보고 있다.새해를 맞아 와이지는 대대적인 ‘세대교체’에 나섰다. 작년 말에는 결별 소식이 줄을 이은 바 있다. 지난달 26일 빅뱅의 태양을 시작으로 27일 빅뱅의 대성이, 30일에는 아이콘 멤버 6인 전원과 배우 강동원이 전속 계약 만료를 알렸다. 하지만 와이지는 새해를 맞아 곧바로 보완책을 제시했다. 지난 1일 공식 유튜브 채널에 공개된 영상에는 회사로 복귀한 양현석 총괄 프로듀서와 새롭게 선보일 걸그룹 베이비몬스터의 모습이 담겼다.“(블랙핑크의) 제니가 여러명 있는 것 같다” 등 호평의 파급력은 컸다. 올해 첫 거래일이었던 지난 2일 와이지는 엔터주 가운데 나홀로 상승하며 9.58%에 달하는 급등세를 보였다.다만 이후 주가는 탄력을 잃었다. 3일 3% 상승세를 이어가는 듯 했으나 다음날 1.4% 반락했고, 5일 횡보세에 이어 6일 오전에는 4% 가까이 급락하고 있다.한 증권사 연구원은 “새 걸그룹에 대한 구체적인 정보는 전혀 없으며 데뷔 시점조차 알려진 게 없다”면서 “향후 대중에게 이들의 매력과 재능을 얼마나 어필하는 지에 따라 장기적인 와이지 주가의 향방이 결정될 것”이라고 말했다.와이지내 가장 존재감이 큰 지드래곤은 회사에 남았지만, 매출 확보와 이익 모멘텀·주가 모멘텀은 각각 별개라는 분석도 나온다. 김현용 현대차증권 연구원은 “빅뱅 멤버 가운데 핵

In [44]:
# 불필요한 내용 삭제 1
# 기자]  이전 내용 삭제
pattern = r'.*기자\]'
filtered_df.loc[filtered_df['WRITER'] == writer, 'PARSED CONTENT'] = \
    filtered_df.loc[filtered_df['WRITER'] == writer, 'PARSED CONTENT'].apply(lambda text: re.sub(pattern, '', text))

In [45]:
# 예시(전처리 후)
filtered_df[filtered_df['WRITER']== writer]["PARSED CONTENT"].iloc[1]

'새 걸그룹 ‘베이비몬스터’ 실체는\n지드래곤 등 아티스트 해외공연\n올 계약 만료 블랙핑크와 재계약\n더블랙레이블과 협력도 과제양현석 총괄 프로듀서 복귀와 새 걸그룹 ‘베이비몬스터’의 등판으로 새해 첫날 10% 가까운 급등세를 보인 와이지엔터테인먼트 주가가 다시 횡보세로 돌아섰다.6일 오전엔 4% 가까운 급락세를 보였다. 아직 정보가 부족한 걸그룹에 대한 기대감만으로 상승세를 이어가기는 역부족인 것으로 풀이된다. 전문가는 베이비몬스터의 성공과 함께 ▷지드래곤을 비롯한 남은 아티스트의 해외공연 ▷블랙핑크와 재계약 ▷관계사인 더블랙레이블과 지속협력 등이 이어져야 주가가 지속적인 탄력을 받을 수 있을 것으로 보고 있다.새해를 맞아 와이지는 대대적인 ‘세대교체’에 나섰다. 작년 말에는 결별 소식이 줄을 이은 바 있다. 지난달 26일 빅뱅의 태양을 시작으로 27일 빅뱅의 대성이, 30일에는 아이콘 멤버 6인 전원과 배우 강동원이 전속 계약 만료를 알렸다. 하지만 와이지는 새해를 맞아 곧바로 보완책을 제시했다. 지난 1일 공식 유튜브 채널에 공개된 영상에는 회사로 복귀한 양현석 총괄 프로듀서와 새롭게 선보일 걸그룹 베이비몬스터의 모습이 담겼다.“(블랙핑크의) 제니가 여러명 있는 것 같다” 등 호평의 파급력은 컸다. 올해 첫 거래일이었던 지난 2일 와이지는 엔터주 가운데 나홀로 상승하며 9.58%에 달하는 급등세를 보였다.다만 이후 주가는 탄력을 잃었다. 3일 3% 상승세를 이어가는 듯 했으나 다음날 1.4% 반락했고, 5일 횡보세에 이어 6일 오전에는 4% 가까이 급락하고 있다.한 증권사 연구원은 “새 걸그룹에 대한 구체적인 정보는 전혀 없으며 데뷔 시점조차 알려진 게 없다”면서 “향후 대중에게 이들의 매력과 재능을 얼마나 어필하는 지에 따라 장기적인 와이지 주가의 향방이 결정될 것”이라고 말했다.와이지내 가장 존재감이 큰 지드래곤은 회사에 남았지만, 매출 확보와 이익 모멘텀·주가 모멘텀은 각각 별개라는 분석도 나온다. 김현용 현대차증권 연구원은 “빅뱅 멤버 가운데 핵

## 뉴 프라임경제

In [46]:
# 예시(전처리 전)
writer = '뉴 프라임경제'  # 언론사 설정 
filtered_df[filtered_df['WRITER']== writer]["PARSED CONTENT"].iloc[0]  # 확인

'[프라임경제] 한증막은 \'한\'주간 \'증\'시가 \'막\'을 내렸다의 줄임말로 즉 국내증시가 한주동안 어떤 요인으로 상승 또는 하락했는지 이유를 살펴본다.이번 주 한증막은 미국의 긴축 우려다. 지난 16일 코스피 지수는 2451.21에 마감했다. 전주 대비로는 0.7% 하락했다. 코스피가 2400선에서 횡보하고 있다. 이번 주는 에스엠(041510) 인수, 위믹스 재상장 등 개별 업종과 종목에서 이벤트가 열렸다.김석환 미래에셋증권 연구원은 "국내증시는 미국의 1월 생산자물가지수(PPI) 지수도 예상치를 상회하면서 인플레이션 부담 확대와 연준 위원들의 50bp 금리 인상 발언까지 더해지며 약세로 마감한 미국 증시 여파가 반영됐다"고 설명했다.이어 "철강, 건설, 기계 등 시크리컬 업종이 상승했다"며 "전날 국내 증시 상승을 이끈 2차전지, 반도체 대형주 전반은 차익실현 매물 출회가 확대됐다"고 덧붙였다.◆연준, 0.50%p 인상 가능성에 시장 \'낙담\'16일(현지시간) 미 노동부는 1월 생산자물가지수(PPI)가 전월보다 0.7%, 전년 동월보다 6.0% 각각 올랐다고 밝혔다. 전월 대비 상승률은 지난해 6월 이후 가장 큰 폭이다. 지난해 12월 0.2% 하락했다가 다시 상승세로 돌아섰다.지난달 PPI는 블룸버그통신이 집계한 전문가 전망치(전월 대비 0.4%·전년 대비 5.4%를 큰 폭 상회했다. 전년 동월 대비로는 상승폭이 12월(6.5%)보다 줄어 7개월 연속 감속했다. 다만 여전히 미 연방준비제도(Fed·연준)의 물가상승률 목표치 2%의 3배에 이른다.로레타 메스터 클리블랜드 연방준비은행 총재는 기자들과 만나 "이전 회의에서 0.50%p 인상의 강력한 근거를 봤다"며 향후 회의에서 0.50%p 인상 가능성을 배제하지 않고 있음을 시사했다.◆尹 "에너지요금 조절·공공요금 동결"지난 15일 오전 윤 대통령은 용산 대통령실에서 제13차 비상경제민생회의 모두발언에서 "난방비 부담이 크게 늘어난 가운데, 교통 등 공공요금 인상 계획 소식까지 더해지면서 (서민과

In [47]:
# 불필요한 내용 삭제 1
filtered_df.loc[filtered_df['WRITER'] == writer, 'PARSED CONTENT'] = \
    filtered_df.loc[filtered_df['WRITER'] == writer, 'PARSED CONTENT'].str.replace("[프라임경제]", "", regex=False)

In [48]:
# 예시(전처리 후)
filtered_df[filtered_df['WRITER']== writer]["PARSED CONTENT"].iloc[0]

' 한증막은 \'한\'주간 \'증\'시가 \'막\'을 내렸다의 줄임말로 즉 국내증시가 한주동안 어떤 요인으로 상승 또는 하락했는지 이유를 살펴본다.이번 주 한증막은 미국의 긴축 우려다. 지난 16일 코스피 지수는 2451.21에 마감했다. 전주 대비로는 0.7% 하락했다. 코스피가 2400선에서 횡보하고 있다. 이번 주는 에스엠(041510) 인수, 위믹스 재상장 등 개별 업종과 종목에서 이벤트가 열렸다.김석환 미래에셋증권 연구원은 "국내증시는 미국의 1월 생산자물가지수(PPI) 지수도 예상치를 상회하면서 인플레이션 부담 확대와 연준 위원들의 50bp 금리 인상 발언까지 더해지며 약세로 마감한 미국 증시 여파가 반영됐다"고 설명했다.이어 "철강, 건설, 기계 등 시크리컬 업종이 상승했다"며 "전날 국내 증시 상승을 이끈 2차전지, 반도체 대형주 전반은 차익실현 매물 출회가 확대됐다"고 덧붙였다.◆연준, 0.50%p 인상 가능성에 시장 \'낙담\'16일(현지시간) 미 노동부는 1월 생산자물가지수(PPI)가 전월보다 0.7%, 전년 동월보다 6.0% 각각 올랐다고 밝혔다. 전월 대비 상승률은 지난해 6월 이후 가장 큰 폭이다. 지난해 12월 0.2% 하락했다가 다시 상승세로 돌아섰다.지난달 PPI는 블룸버그통신이 집계한 전문가 전망치(전월 대비 0.4%·전년 대비 5.4%를 큰 폭 상회했다. 전년 동월 대비로는 상승폭이 12월(6.5%)보다 줄어 7개월 연속 감속했다. 다만 여전히 미 연방준비제도(Fed·연준)의 물가상승률 목표치 2%의 3배에 이른다.로레타 메스터 클리블랜드 연방준비은행 총재는 기자들과 만나 "이전 회의에서 0.50%p 인상의 강력한 근거를 봤다"며 향후 회의에서 0.50%p 인상 가능성을 배제하지 않고 있음을 시사했다.◆尹 "에너지요금 조절·공공요금 동결"지난 15일 오전 윤 대통령은 용산 대통령실에서 제13차 비상경제민생회의 모두발언에서 "난방비 부담이 크게 늘어난 가운데, 교통 등 공공요금 인상 계획 소식까지 더해지면서 (서민과 취약계층이)

## 뉴스웨이

In [49]:
# 예시(전처리 전)
writer = '뉴스웨이'  # 언론사 설정 
filtered_df[filtered_df['WRITER']== writer]["PARSED CONTENT"].iloc[0]  # 확인

'하이브가 에스엠의 경영권 확보를 위한 지분 인수에 나서면서 에스엠 주가가 12%대 강세를 보이고 있다. \\n \\n10일 오전 9시15분 기준 에스엠은 전 거래일 대비 12.18%(1만2000원) 오른 11만500원에 거래되고 있다.  \\n \\n이날 하이브는 이수만 SM 대주주가 보유한 지분 14.8%를 4228억 원에 인수한다고 공시했다. 주당 가격은 12만 원이다. SM 1대 주주인 이수만의 지분율은 18.46%로, 하이브는 이번 거래로 최대주주에 오르게 됐다. \\n \\n한편 같은 시간 하이브는 전 거래일 대비 3.88%(7700원) 오른 20만6000원에 거래되고 있다. \\n\\n임주희 기자 ljh@newsway.co.kr\\n\\n<온라인 경제미디어 뉴스웨이 - 무단전재 & 재배포 금지>                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        

In [50]:
# 불필요한 내용 삭제 1
filtered_df.loc[filtered_df['WRITER'] == writer, 'PARSED CONTENT'] = \
    filtered_df.loc[filtered_df['WRITER'] == writer, 'PARSED CONTENT'].str.replace("@newsway.co.kr\n\n<온라인 경제미디어 뉴스웨이 - 무단전재 & 재배포 금지>", "", regex=False)

In [51]:
# 예시(전처리 후)
filtered_df[filtered_df['WRITER']== writer]["PARSED CONTENT"].iloc[0]

'하이브가 에스엠의 경영권 확보를 위한 지분 인수에 나서면서 에스엠 주가가 12%대 강세를 보이고 있다. \\n \\n10일 오전 9시15분 기준 에스엠은 전 거래일 대비 12.18%(1만2000원) 오른 11만500원에 거래되고 있다.  \\n \\n이날 하이브는 이수만 SM 대주주가 보유한 지분 14.8%를 4228억 원에 인수한다고 공시했다. 주당 가격은 12만 원이다. SM 1대 주주인 이수만의 지분율은 18.46%로, 하이브는 이번 거래로 최대주주에 오르게 됐다. \\n \\n한편 같은 시간 하이브는 전 거래일 대비 3.88%(7700원) 오른 20만6000원에 거래되고 있다. \\n\\n임주희 기자 ljh@newsway.co.kr\\n\\n<온라인 경제미디어 뉴스웨이 - 무단전재 & 재배포 금지>                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        

## IR 큐더스 

In [52]:
# 예시(전처리 전)
writer = 'IR큐더스'  # 언론사 설정 
filtered_df[filtered_df['WRITER']== writer]["PARSED CONTENT"].iloc[0]  # 확인

'[보도자료] 자이언트스텝, ‘2022 앤어워드’ 디지털 미디어＆서비스 부문 최고상 수상\xa0▶ 디지털 미디어&서비스 부문 최고상인 과기부 장관상 포함 3관왕 달성_x000D_▶ ‘디페스타 XR콘서트’, ‘지구의 시간 전시회’ 등 고도화된 확장 현실 콘텐츠 구현_x000D_\xa0_x000D_<2023-01-16> 리얼타임 콘텐츠 솔루션 기업 자이언트스텝(289220)이 ‘2022 앤어워드’ 디지털 미디어&서비스 부문 최고상인 과학기술통신부 장관상을 수상했다고 16일 밝혔다. 자이언트스텝은 지난해 앤어워드 2관왕에 이어 올해에는 3관왕을 달성하는 영예를 안게 됐다._x000D__x000D_   _x000D__x000D_자이언트스텝이 제작한 ‘디페스타 XR 콘서트(D′FESTA: XR CONCERT)′ 콘텐츠가 ‘디지털 미디어 & 서비스’ 부문 출품작 중 최고 점수를 획득해 최고상인 과학기술정보통신부 장관상을 수상했다. 이밖에도 ′지구의 시간 전시회′, ′광주미디어아트플랫폼′ 콘텐츠가 ′디지털 미디어 & 서비스’ 정부/공공/지자체 부문 그랑프리와 위너로 각각 선정됐다._x000D__x000D_   _x000D__x000D_최고상 수상작 ‘디페스타 XR콘서트’는 BTS, 트와이스, 세븐틴, NCT127 등 아티스트들의 무대를 확장 현실(XR)로 즐길 수 있게 제작한 콘텐츠이다. 가상 공연장을 제작해 실제 오프라인 콘서트 같은 실황 공연을 연출했으며, 아티스트들의 무대를 실제 공연처럼 느낄 만큼 고도화된 확장 현실 콘텐츠를 제작했다. 또한, 같이 수상한 ′지구의 시간 전시회′, ′광주미디어아트플랫폼′은 몰입감 있는 인터랙티브 미디어아트 콘텐츠를 구현했다는 평을 받고 있다._x000D__x000D_   _x000D__x000D_앤어워드(Awards for New Digital Awards)는 한 해 동안 디지털산업에 기여한 작품들을 공정한 심사를 통해 수상하는 국내 최대의 디지털 분야 어워드 행사로 올해로 16회째를 맞이했다. 한국디지털기업협회가 주관하고 과학기술정

In [53]:
# 불필요한 내용 삭제 1
filtered_df.loc[filtered_df['WRITER'] == writer, 'PARSED CONTENT'] = \
    filtered_df.loc[filtered_df['WRITER'] == writer, 'PARSED CONTENT'].str.replace("본 내용은 기업에 대한 이해 증진을 위한 목적으로, 투자 권유를 목적으로 한 것이 아닙니다.투자에 관한 결정은 투자자 본인에게 있으며, 회사는 투자에 관해 일체의 책임을 지지 않습니다.", "", regex=False)

In [54]:
# 불필요한 내용 삭제 2
filtered_df.loc[filtered_df['WRITER'] == writer, 'PARSED CONTENT'] = \
    filtered_df.loc[filtered_df['WRITER'] == writer, 'PARSED CONTENT'].str.replace("Q&A시간 : 10시 ~ 12시", "", regex=False)

In [55]:
# 불필요한 내용 삭제 3
filtered_df.loc[filtered_df['WRITER'] == writer, 'PARSED CONTENT'] = \
    filtered_df.loc[filtered_df['WRITER'] == writer, 'PARSED CONTENT'].str.replace("IR Page에 접속하시면 다양한 투자 정보를 확인할 수 있습니다.", "", regex=False)

In [56]:
# 예시(전처리 후)
filtered_df[filtered_df['WRITER']== writer]["PARSED CONTENT"].iloc[0]

'[보도자료] 자이언트스텝, ‘2022 앤어워드’ 디지털 미디어＆서비스 부문 최고상 수상\xa0▶ 디지털 미디어&서비스 부문 최고상인 과기부 장관상 포함 3관왕 달성_x000D_▶ ‘디페스타 XR콘서트’, ‘지구의 시간 전시회’ 등 고도화된 확장 현실 콘텐츠 구현_x000D_\xa0_x000D_<2023-01-16> 리얼타임 콘텐츠 솔루션 기업 자이언트스텝(289220)이 ‘2022 앤어워드’ 디지털 미디어&서비스 부문 최고상인 과학기술통신부 장관상을 수상했다고 16일 밝혔다. 자이언트스텝은 지난해 앤어워드 2관왕에 이어 올해에는 3관왕을 달성하는 영예를 안게 됐다._x000D__x000D_   _x000D__x000D_자이언트스텝이 제작한 ‘디페스타 XR 콘서트(D′FESTA: XR CONCERT)′ 콘텐츠가 ‘디지털 미디어 & 서비스’ 부문 출품작 중 최고 점수를 획득해 최고상인 과학기술정보통신부 장관상을 수상했다. 이밖에도 ′지구의 시간 전시회′, ′광주미디어아트플랫폼′ 콘텐츠가 ′디지털 미디어 & 서비스’ 정부/공공/지자체 부문 그랑프리와 위너로 각각 선정됐다._x000D__x000D_   _x000D__x000D_최고상 수상작 ‘디페스타 XR콘서트’는 BTS, 트와이스, 세븐틴, NCT127 등 아티스트들의 무대를 확장 현실(XR)로 즐길 수 있게 제작한 콘텐츠이다. 가상 공연장을 제작해 실제 오프라인 콘서트 같은 실황 공연을 연출했으며, 아티스트들의 무대를 실제 공연처럼 느낄 만큼 고도화된 확장 현실 콘텐츠를 제작했다. 또한, 같이 수상한 ′지구의 시간 전시회′, ′광주미디어아트플랫폼′은 몰입감 있는 인터랙티브 미디어아트 콘텐츠를 구현했다는 평을 받고 있다._x000D__x000D_   _x000D__x000D_앤어워드(Awards for New Digital Awards)는 한 해 동안 디지털산업에 기여한 작품들을 공정한 심사를 통해 수상하는 국내 최대의 디지털 분야 어워드 행사로 올해로 16회째를 맞이했다. 한국디지털기업협회가 주관하고 과학기술정

## 매일 경제 

In [57]:
# 예시(전처리 전)
writer = '매일경제'  # 언론사 설정 
filtered_df[filtered_df['WRITER']== writer]["PARSED CONTENT"].iloc[0]  # 확인

'증권사들이 작년 데뷔한 신인 걸그룹 뉴진스의 빌보드 핫100 차트 진입 등을 계기로 하이브의 목표주가를 잇따라 올렸다.하이투자증권은 20일 낸 리포트에서 뉴진스의 글로벌 성과와 위버스 비즈니스모델 고도화 등을 고려해 목표 주가수익비율(PER) 39배를 적용, 하이브의 목표주가를 기존 19만2000천원에서 23만원으로 올렸다. 2018년 기준 엔터테인먼트사 평균 PER은 30배지만 하이브의 기업 가치를 고려하면 더 높은 PER 적용이 필요하다는 분석이다.하이투자증권은 “뉴진스는 국내에서만 아니라 미국, 일본 등 글로벌 주요 음악시장에서 K팝 기준 최소 3위 이상의 성과를 보여주고 있다”며 “뉴진스의 스포티파이 스트리밍 성적은 트와이스를 뛰어넘고 블랙핑크에 근접하는 수준”이라고 설명했다.그러면서 “뉴진스의 스포티파이 청취자 수 성장 추세가 과거 블랙핑크의 성장 패턴을 압축적으로 보여준다”면서 “주요 시장인 미국 등에서 특히 반응이 좋아 블랙핑크보다 이른 시간 안에 블랙핑크 이상의 입지를 구축할 가능성이 높다”고 말했다.메리츠증권도 하이브에 대해 보이그룹 TXT와 세븐틴의 콘서트 규모 확대, 위버스의 구독 모델 도입, 뉴진스의 빠른 성장세 등으로 호실적이 예상된다며 투자의견을 ‘매수’로 유지, 목표주가는 기존 17만원에서 21만원으로 상향했다.이 외에 IBK투자증권(18만원→20만원), 다올투자증권(15만원→23만원), 한화투자증권(18만5000원→20만5000원) 등도 이달 들어 목표가를 높게 조정했다._x000D_\n_x000D_\n_x000D_\n_x000D_\n                                                                                                                                                                                                                                 

In [58]:
# 불필요한 내용 삭제 1
# [오거스타  이후내용 삭제 
pattern = r'\[오거스타[^\]]*\]'
filtered_df.loc[filtered_df['WRITER'] == writer, 'PARSED CONTENT'] = \
    filtered_df.loc[filtered_df['WRITER'] == writer, 'PARSED CONTENT'].apply(lambda text: re.sub(pattern, '', text))

In [59]:
# 불필요한 내용 삭제 2
# [ⓒ 이후내용 삭제 
pattern = r'\[ⓒ[^\]]*\]'
filtered_df.loc[filtered_df['WRITER'] == writer, 'PARSED CONTENT'] = \
    filtered_df.loc[filtered_df['WRITER'] == writer, 'PARSED CONTENT'].apply(lambda text: re.sub(pattern, '', text))


In [60]:
# 예시(전처리 후)
filtered_df[filtered_df['WRITER']== writer]["PARSED CONTENT"].iloc[0]

'증권사들이 작년 데뷔한 신인 걸그룹 뉴진스의 빌보드 핫100 차트 진입 등을 계기로 하이브의 목표주가를 잇따라 올렸다.하이투자증권은 20일 낸 리포트에서 뉴진스의 글로벌 성과와 위버스 비즈니스모델 고도화 등을 고려해 목표 주가수익비율(PER) 39배를 적용, 하이브의 목표주가를 기존 19만2000천원에서 23만원으로 올렸다. 2018년 기준 엔터테인먼트사 평균 PER은 30배지만 하이브의 기업 가치를 고려하면 더 높은 PER 적용이 필요하다는 분석이다.하이투자증권은 “뉴진스는 국내에서만 아니라 미국, 일본 등 글로벌 주요 음악시장에서 K팝 기준 최소 3위 이상의 성과를 보여주고 있다”며 “뉴진스의 스포티파이 스트리밍 성적은 트와이스를 뛰어넘고 블랙핑크에 근접하는 수준”이라고 설명했다.그러면서 “뉴진스의 스포티파이 청취자 수 성장 추세가 과거 블랙핑크의 성장 패턴을 압축적으로 보여준다”면서 “주요 시장인 미국 등에서 특히 반응이 좋아 블랙핑크보다 이른 시간 안에 블랙핑크 이상의 입지를 구축할 가능성이 높다”고 말했다.메리츠증권도 하이브에 대해 보이그룹 TXT와 세븐틴의 콘서트 규모 확대, 위버스의 구독 모델 도입, 뉴진스의 빠른 성장세 등으로 호실적이 예상된다며 투자의견을 ‘매수’로 유지, 목표주가는 기존 17만원에서 21만원으로 상향했다.이 외에 IBK투자증권(18만원→20만원), 다올투자증권(15만원→23만원), 한화투자증권(18만5000원→20만5000원) 등도 이달 들어 목표가를 높게 조정했다._x000D_\n_x000D_\n_x000D_\n_x000D_\n                                                                                                                                                                                                                                 

## 연합 인포 해외 → 해당 언론사 기사 없음(필터링 과정에서 제거됨)

## 한경 WOW 해외 → 해당 언론사 기사 없음(필터링 과정에서 제거됨)

## CNBC(AI뉴스) → 해당 언론사 기사 없음(필터링 과정에서 제거됨)


## 아시아경제

In [61]:
# 예시(전처리 전)
writer = '아시아경제'  # 언론사 설정 
filtered_df[filtered_df['WRITER']== writer]["PARSED CONTENT"].iloc[0]  # 확인

'[아시아경제 이민지 기자] NH투자증권은 19일 와이지엔터테인먼트에 대해 투자의견 매수와 목표주가 7만7000원을 유지했다. 블랙핑크의 재계약 불확실성이 남아있지만, 신인 걸그룹 론칭과 GD의 솔로 활동이 주가 상승을 끌어낼 것으로 예상했다.\n\n와이지엔터테인먼트는 지난해 12월 말 양현석 총괄 관련 이슈가 해소된 이후 사업 전반에 속도가 붙고 있다. 일부 아티스트 라인업을 정리하고 주려 아티스트 중심의 사업 진행에 박차를 가하고 있다. 1월 초에는 신인 걸그룹인 베이비몬스터 론칭 티저가 공개됐고, GD의 경우 직접 본인의 연내 솔로 활동 재개 의지를 밝혔다.\n이화정 NH투자증권 연구원은 “베이비몬스터의 경우 멤버들의 퍼포먼스 티저 영상이 매주 공개되고 있는데 지금까지 대중 반응은 우호적이다”며 “신인에 GD의 지원사격까지 더해지며 단일 아티스트 의존도가 해소될 것”이라고 설명했다.\n블랙핑크 재계약 이슈로 주가는 저평가를 받고 있지만, 부정적인 이슈를 미리 고려할 필요성은 없을 것으로 보인다. 블랙핑크의 계약 만기일은 올해 8월이다. 이화정 연구원은 “아티스트와 기업 양쪽의 실익을 고려할 때 재계약이 무난히 진행될 가능성이 높다”며 “과도한 우려는 내려놓을 시점”이라고 분석했다.\n4분기 예상 매출액은 1455억원, 영업이익은 23억원으로 전년동기대비 각각 64%, 74% 성장할 것으로 추정된다. 블랙핑크 월드투어 관련 티켓과 MD 수익 반영됐기 때문이다. 트레저의 일본투어 관련 수익도 인식될 것으로 예상된다.\n\n[관련기사]\n☞ \n☞ \n☞ \n☞ \n☞ \n\xa0\xa0\n\n이민지 기자 ming@asiae.co.kr\n<ⓒ경제를 보는 눈, 세계를 보는 창 아시아경제(www.asiae.co.kr) 무단전재 배포금지>\n                                                                                                                               

In [62]:
# 불필요한 내용 삭제 1
# [관련기사] 이후내용 삭제
pattern = r'\[관련기사[^\]]*\].*'
filtered_df.loc[filtered_df['WRITER'] == writer, 'PARSED CONTENT'] = \
    filtered_df.loc[filtered_df['WRITER'] == writer, 'PARSED CONTENT'].apply(lambda text: re.sub(pattern, '', text, flags=re.DOTALL))


In [63]:
# 예시(전처리 후)
filtered_df[filtered_df['WRITER']== writer]["PARSED CONTENT"].iloc[0]

'[아시아경제 이민지 기자] NH투자증권은 19일 와이지엔터테인먼트에 대해 투자의견 매수와 목표주가 7만7000원을 유지했다. 블랙핑크의 재계약 불확실성이 남아있지만, 신인 걸그룹 론칭과 GD의 솔로 활동이 주가 상승을 끌어낼 것으로 예상했다.\n\n와이지엔터테인먼트는 지난해 12월 말 양현석 총괄 관련 이슈가 해소된 이후 사업 전반에 속도가 붙고 있다. 일부 아티스트 라인업을 정리하고 주려 아티스트 중심의 사업 진행에 박차를 가하고 있다. 1월 초에는 신인 걸그룹인 베이비몬스터 론칭 티저가 공개됐고, GD의 경우 직접 본인의 연내 솔로 활동 재개 의지를 밝혔다.\n이화정 NH투자증권 연구원은 “베이비몬스터의 경우 멤버들의 퍼포먼스 티저 영상이 매주 공개되고 있는데 지금까지 대중 반응은 우호적이다”며 “신인에 GD의 지원사격까지 더해지며 단일 아티스트 의존도가 해소될 것”이라고 설명했다.\n블랙핑크 재계약 이슈로 주가는 저평가를 받고 있지만, 부정적인 이슈를 미리 고려할 필요성은 없을 것으로 보인다. 블랙핑크의 계약 만기일은 올해 8월이다. 이화정 연구원은 “아티스트와 기업 양쪽의 실익을 고려할 때 재계약이 무난히 진행될 가능성이 높다”며 “과도한 우려는 내려놓을 시점”이라고 분석했다.\n4분기 예상 매출액은 1455억원, 영업이익은 23억원으로 전년동기대비 각각 64%, 74% 성장할 것으로 추정된다. 블랙핑크 월드투어 관련 티켓과 MD 수익 반영됐기 때문이다. 트레저의 일본투어 관련 수익도 인식될 것으로 예상된다.\n\n'

## 컨슈머타임스

In [64]:
# 예시(전처리 전)
writer = '컨슈머타임스'  # 언론사 설정 
filtered_df[filtered_df['WRITER']== writer]["PARSED CONTENT"].iloc[1]  # 확인

"_x000D_\n_x000D_\n[사진= 물고기컴퍼니, 빅히트 뮤직, SM엔터테인먼트 제공] _x000D_\n_x000D_\n_x000D_\n_x000D_\n컨슈머타임스=김동역 기자 | '더팩트 뮤직 어워즈(TMA)'가 'TMA 베스트 뮤직' 부문을 신설했다. \n_x000D_\n_x000D_\n2일부터 16일까지 2주 동안 팬앤스타에서 '더팩트 뮤직 어워즈'의 'TMA 베스트 뮤직 - 겨울' 부문 온라인 사전 투표가 진행된다.\n_x000D_\n_x000D_\n결선 투표는 오는 16일부터 30일까지 2주간 이어진다. \n_x000D_\n_x000D_\n'TMA 베스트 뮤직'은 분기별로 음원을 발매한 아티스트를 대상으로 하는 투표다. 이번에 진행되는 'TMA 베스트 뮤직 - 겨울' 부문에는 2022년 10월부터 12월까지 OST를 제외하고 음원을 발매한 모든 아티스트를 대상으로 투표를 진행한다. \n_x000D_\n_x000D_\n이에 방탄소년단 진RM, 임영웅, 김호중, 장민호, NCT 드림, 스트레이 키즈, 더보이즈, 에이티즈, ITZY, 트레저, 르세라핌, 뉴진스, 강다니엘, 황치열, 마마무, 하이라이트, 엔플라잉, 케플러, 백호, 포레스텔라, 박지훈, 빅톤, 업텐션, 레드벨벳, 보아, 베리베리 등 기간 내 음원을 발매한 다채로운 아티스트가 후보로 올랐다. \n_x000D_\n_x000D_\n투표는 온라인 종합 매체 더팩트가 운영하는 아이돌 전문 종합 서비스 팬앤스타를 통해 진행되며 '일반별'과 '무지개별', 동영상 광고를 시청해 참여하는 '동영상 투표'까지 세 가지 방식으로 투표할 수 있다.\n"

In [65]:
# 불필요한 내용 삭제 1
# 컨슈머타임스= 이전내용 모두삭제
pattern = r'.*컨슈머타임스='
filtered_df.loc[filtered_df['WRITER'] == writer, 'PARSED CONTENT'] = \
    filtered_df.loc[filtered_df['WRITER'] == writer, 'PARSED CONTENT'].apply(lambda text: re.sub(pattern, '', text, flags=re.DOTALL))


In [66]:
# 예시(전처리 후)
filtered_df[filtered_df['WRITER']== writer]["PARSED CONTENT"].iloc[1]

"김동역 기자 | '더팩트 뮤직 어워즈(TMA)'가 'TMA 베스트 뮤직' 부문을 신설했다. \n_x000D_\n_x000D_\n2일부터 16일까지 2주 동안 팬앤스타에서 '더팩트 뮤직 어워즈'의 'TMA 베스트 뮤직 - 겨울' 부문 온라인 사전 투표가 진행된다.\n_x000D_\n_x000D_\n결선 투표는 오는 16일부터 30일까지 2주간 이어진다. \n_x000D_\n_x000D_\n'TMA 베스트 뮤직'은 분기별로 음원을 발매한 아티스트를 대상으로 하는 투표다. 이번에 진행되는 'TMA 베스트 뮤직 - 겨울' 부문에는 2022년 10월부터 12월까지 OST를 제외하고 음원을 발매한 모든 아티스트를 대상으로 투표를 진행한다. \n_x000D_\n_x000D_\n이에 방탄소년단 진RM, 임영웅, 김호중, 장민호, NCT 드림, 스트레이 키즈, 더보이즈, 에이티즈, ITZY, 트레저, 르세라핌, 뉴진스, 강다니엘, 황치열, 마마무, 하이라이트, 엔플라잉, 케플러, 백호, 포레스텔라, 박지훈, 빅톤, 업텐션, 레드벨벳, 보아, 베리베리 등 기간 내 음원을 발매한 다채로운 아티스트가 후보로 올랐다. \n_x000D_\n_x000D_\n투표는 온라인 종합 매체 더팩트가 운영하는 아이돌 전문 종합 서비스 팬앤스타를 통해 진행되며 '일반별'과 '무지개별', 동영상 광고를 시청해 참여하는 '동영상 투표'까지 세 가지 방식으로 투표할 수 있다.\n"

## eDaily

In [67]:
# 예시(전처리 전)
writer = 'eDaily'  # 언론사 설정 
filtered_df[filtered_df['WRITER']== writer]["PARSED CONTENT"].iloc[0]  # 확인

'- KB증권 “군 입대 BTS에 도전 이어질 것”_x000D_\n- “스트레이 키즈, 군 이슈 없고 이미 500만장 돌파”_x000D_\n_x000D_\n[이데일리 최훈길 기자] 방탄소년단(BTS)을 이을 아이돌 그룹에 스트레이 키즈가 유력하다는 전망이 제기됐다. _x000D_\n_x000D_\n이선화 KB증권 애널리스트와 류은애 연구원은 2일 ‘연차별 판매량으로 본 2023 유망주’ 보고서에서 “BTS의 군 입대로 인해 BTS의 완전체 활동이 불가능해지면서 2023년에는 K팝의 역사를 써 내려온 BTS의 기록을 깨려는 보이그룹들의 도전이 이어질 것”이라며 “넥스트 BTS로 가장 유망한 아티스트는 스트레이 키즈”라고 밝혔다. _x000D_\n_x000D_\n스트레이 키즈. (사진=이데일리DB)_x000D_\n_x000D_\n스트레이 키즈는 지난 10월7일 발매한 미니 앨범 7집 맥시던트(Maxident)로 누적 판매량 300만장을 돌파했다. 단일 앨범 총 판매량이 300만장을 넘은 아티스트는 BTS와 스트레이 키즈가 유일하다. _x000D_\n_x000D_\n이 애널리스트는 “스트레이 키즈는 2022년 544만장(11월까지 집계)의 앨범을 판매하면서 BTS 다음으로 앨범을 많이 판매한 아티스트로 등극했다”며 “스트레이 키즈 멤버의 첫 군입대는 2026년으로 군 입대 리스크로부터도 당분간 자유롭다”고 지적했다.  _x000D_\n_x000D_\n엔시티 드림(NCT DREAM)이 스트레이 키즈의 뒤를 바짝 쫓고 있다. NCT DREAM은 지난해 누적 앨범 판매량이 409만장을 기록했다. 그룹 투모로우바이투게더(TXT_는 데뷔 3년차에, 엔하이픈은 데뷔 2년차에 200만장 이상의 판매고를 달성했다._x000D_\n_x000D_\n이 애널리스트는 “NCT DREAM은 성장 속도가 가파르게 둔화되고 있는 것이 아쉽다”며 “TXT와 엔하이픈은 성장 속도가 놀랍지만, 절대적인 판매량에 있어 스트레이 키즈와 2배 이상 차이가 나 넥스트 BTS가 되기에는 시간이 필요하다”고

In [68]:
# 불필요한 내용 삭제 1
# 종합 경제정보 미디어 이데일리  이후내용 삭제 
pattern = r'종합 경제정보 미디어 이데일리.*'
filtered_df.loc[filtered_df['WRITER'] == writer, 'PARSED CONTENT'] = \
    filtered_df.loc[filtered_df['WRITER'] == writer, 'PARSED CONTENT'].apply(lambda text: re.sub(pattern, '', text) if isinstance(text, str) else text)


In [69]:
# 예시(전처리 후)
filtered_df[filtered_df['WRITER']== writer]["PARSED CONTENT"].iloc[0]

'- KB증권 “군 입대 BTS에 도전 이어질 것”_x000D_\n- “스트레이 키즈, 군 이슈 없고 이미 500만장 돌파”_x000D_\n_x000D_\n[이데일리 최훈길 기자] 방탄소년단(BTS)을 이을 아이돌 그룹에 스트레이 키즈가 유력하다는 전망이 제기됐다. _x000D_\n_x000D_\n이선화 KB증권 애널리스트와 류은애 연구원은 2일 ‘연차별 판매량으로 본 2023 유망주’ 보고서에서 “BTS의 군 입대로 인해 BTS의 완전체 활동이 불가능해지면서 2023년에는 K팝의 역사를 써 내려온 BTS의 기록을 깨려는 보이그룹들의 도전이 이어질 것”이라며 “넥스트 BTS로 가장 유망한 아티스트는 스트레이 키즈”라고 밝혔다. _x000D_\n_x000D_\n스트레이 키즈. (사진=이데일리DB)_x000D_\n_x000D_\n스트레이 키즈는 지난 10월7일 발매한 미니 앨범 7집 맥시던트(Maxident)로 누적 판매량 300만장을 돌파했다. 단일 앨범 총 판매량이 300만장을 넘은 아티스트는 BTS와 스트레이 키즈가 유일하다. _x000D_\n_x000D_\n이 애널리스트는 “스트레이 키즈는 2022년 544만장(11월까지 집계)의 앨범을 판매하면서 BTS 다음으로 앨범을 많이 판매한 아티스트로 등극했다”며 “스트레이 키즈 멤버의 첫 군입대는 2026년으로 군 입대 리스크로부터도 당분간 자유롭다”고 지적했다.  _x000D_\n_x000D_\n엔시티 드림(NCT DREAM)이 스트레이 키즈의 뒤를 바짝 쫓고 있다. NCT DREAM은 지난해 누적 앨범 판매량이 409만장을 기록했다. 그룹 투모로우바이투게더(TXT_는 데뷔 3년차에, 엔하이픈은 데뷔 2년차에 200만장 이상의 판매고를 달성했다._x000D_\n_x000D_\n이 애널리스트는 “NCT DREAM은 성장 속도가 가파르게 둔화되고 있는 것이 아쉽다”며 “TXT와 엔하이픈은 성장 속도가 놀랍지만, 절대적인 판매량에 있어 스트레이 키즈와 2배 이상 차이가 나 넥스트 BTS가 되기에는 시간이 필요하다”고

## 글로벌이코노믹

In [70]:
# 예시(전처리 전)
writer = '글로벌이코노믹'  # 언론사 설정 
filtered_df[filtered_df['WRITER']== writer]["PARSED CONTENT"].iloc[0]  # 확인

'아르헨티나의 리튬 공장에서 원료를 취급하는 모습. 사진=로이터 통신 중국 전기차 판매 둔화 소식에 애널리스트들과 원자재 전문가들이 리튬 공급 과잉으로 리튬 가격이 급락할 것을 19일(현지 시간) 경고했다.원자재 광부부터 배터리 제조기업 및 자동차 기업에 이르기까지 중국 전기차(EV) 공급망에 포함된 기업들은 EV열풍에 힘입어 주가가 상승했다. 그 중 중국 리튬 채굴자들은 지난해 크게 상승한 리튬 가격으로 순익이 급증하면서 줄줄히 어닝 서프라이즈를 기록했다. 중국의 기존 리튬 채굴 기업인 톈치리튬과 간펑리튬 등은 지난해 리튬 가격 급등으로 순이익이 2020년 대비 각각 29배, 5배 증가했다. 리튬은 전기 및 하이브리드 자동차에 필수적으로 필요한 배터리를 만드는 핵심 소재다. 그러나 리튬 광산은 규모가 큰 편이 드물고 지각 전체에 소량 흩어져 있어 금속 추출이 힘들거나 환경 오염이 심한 경우가 많다. 그러나 리튬 채굴의 높은 수익이 알려지면서 기존 광산기업을 비롯해 배터리 기업, EV기업을 포함해 리튬 채굴과 직접적인 관련이 없는 기업들까지 리튬 채굴에 대규모 투자를 하기 시작했다. 지난해 11월 중국에서만 리튬사업과는 상관이 없는 심양 취화 장신구, 부동산 개발기업인 선디랜드 인베스트먼트 등 4개 기업이 대규모 리튬 채굴 투자를 진행했다. 이러한 대규모 투자를 바탕으로 리튬 가격은 지난해 말부터 하락세로 접어들었다. 국제 리튬 가격은 영국 런던금속거래소(LME)에서 13일 기준 톤당 44만7500위안(약 8240만원)으로 거래돼 6개월 만의 최저치를 기록했다. 전월 평균 대비 14.05% 하락한 값이다. 톤당 58만1500위안을 기록했던 지난해 11월과 비교하면 약 23% 하락했다.애널리스트들은 리튬 채굴에 투자하는 자금이 기하 급수적으로 증가한 것을 감안할 때 리튬 공급이 2023년에 수요를 따라잡고 과잉공급으로 넘어갈 것을       전망하고 있다. 민메탈증권의 애널리스트는 올해 리튬 공급이 전 세계적으로 약 33% 증가했다고 추정했다. 절강 증권의 애널

In [71]:
# 불필요한 내용 삭제 1
# 글로벌이코노믹 기자 이후내용 삭제 
pattern = r'글로벌이코노믹 기자.*'
filtered_df.loc[filtered_df['WRITER'] == writer, 'PARSED CONTENT'] = \
    filtered_df.loc[filtered_df['WRITER'] == writer, 'PARSED CONTENT'].apply(lambda text: re.sub(pattern, '', text))


In [72]:
# 예시(전처리 후)
filtered_df[filtered_df['WRITER']== writer]["PARSED CONTENT"].iloc[0]

'아르헨티나의 리튬 공장에서 원료를 취급하는 모습. 사진=로이터 통신 중국 전기차 판매 둔화 소식에 애널리스트들과 원자재 전문가들이 리튬 공급 과잉으로 리튬 가격이 급락할 것을 19일(현지 시간) 경고했다.원자재 광부부터 배터리 제조기업 및 자동차 기업에 이르기까지 중국 전기차(EV) 공급망에 포함된 기업들은 EV열풍에 힘입어 주가가 상승했다. 그 중 중국 리튬 채굴자들은 지난해 크게 상승한 리튬 가격으로 순익이 급증하면서 줄줄히 어닝 서프라이즈를 기록했다. 중국의 기존 리튬 채굴 기업인 톈치리튬과 간펑리튬 등은 지난해 리튬 가격 급등으로 순이익이 2020년 대비 각각 29배, 5배 증가했다. 리튬은 전기 및 하이브리드 자동차에 필수적으로 필요한 배터리를 만드는 핵심 소재다. 그러나 리튬 광산은 규모가 큰 편이 드물고 지각 전체에 소량 흩어져 있어 금속 추출이 힘들거나 환경 오염이 심한 경우가 많다. 그러나 리튬 채굴의 높은 수익이 알려지면서 기존 광산기업을 비롯해 배터리 기업, EV기업을 포함해 리튬 채굴과 직접적인 관련이 없는 기업들까지 리튬 채굴에 대규모 투자를 하기 시작했다. 지난해 11월 중국에서만 리튬사업과는 상관이 없는 심양 취화 장신구, 부동산 개발기업인 선디랜드 인베스트먼트 등 4개 기업이 대규모 리튬 채굴 투자를 진행했다. 이러한 대규모 투자를 바탕으로 리튬 가격은 지난해 말부터 하락세로 접어들었다. 국제 리튬 가격은 영국 런던금속거래소(LME)에서 13일 기준 톤당 44만7500위안(약 8240만원)으로 거래돼 6개월 만의 최저치를 기록했다. 전월 평균 대비 14.05% 하락한 값이다. 톤당 58만1500위안을 기록했던 지난해 11월과 비교하면 약 23% 하락했다.애널리스트들은 리튬 채굴에 투자하는 자금이 기하 급수적으로 증가한 것을 감안할 때 리튬 공급이 2023년에 수요를 따라잡고 과잉공급으로 넘어갈 것을       전망하고 있다. 민메탈증권의 애널리스트는 올해 리튬 공급이 전 세계적으로 약 33% 증가했다고 추정했다. 절강 증권의 애널

## 데이터투자

In [73]:
# 예시(전처리 전)
writer = '데이터투자'  # 언론사 설정 
filtered_df[filtered_df['WRITER']== writer]["PARSED CONTENT"].iloc[0]  # 확인

'"에스엠, NCT DREAM과 에스파 성장 지속"-교보證"목표가 15%↑ 12만원 제시"-교보證  교보증권은 9일 에스엠에 대한 투자의견을 매수, 목표주가를 12만원으로 제시했다. 이날 제시한 목표주가는 이전에 교보증권이 제시한 목표주가 10만4000원 대비 15% 높은 수준이다.교보증권 박성국 애널리스트가 작성한 에스엠 리포트 주요내용은 아래와 같다.▶ EXO 백현과 샤이니 태민 전역으로 활동 라인업 최대▶ NCT DREAM과 에스파 성장 지속▶ 4Q22 Preview▶ FY23 Preview: 영업이익 YoY +60.4% 예상, Top-picks로 새롭게 제시권희진 데이터투자 기자 news@datatooza.com추천 콘텐츠이 컨텐츠는 투자 참고용으로 이를 근거로 한 투자 손실에 대해 책임을 지지 않습니다.<저작권자 ©데이터투자(www.datatooza.com), 무단전재 및 재배포금지>                                                                                    '

In [74]:
# 불필요한 내용 삭제 1
# 데이터투자 기자 이후내용 삭제 
pattern = r'데이터투자 기자.*'
filtered_df.loc[filtered_df['WRITER'] == writer, 'PARSED CONTENT'] = \
    filtered_df.loc[filtered_df['WRITER'] == writer, 'PARSED CONTENT'].apply(lambda text: re.sub(pattern, '', text))


In [75]:
# 예시(전처리 후)
filtered_df[filtered_df['WRITER']== writer]["PARSED CONTENT"].iloc[0]

'"에스엠, NCT DREAM과 에스파 성장 지속"-교보證"목표가 15%↑ 12만원 제시"-교보證  교보증권은 9일 에스엠에 대한 투자의견을 매수, 목표주가를 12만원으로 제시했다. 이날 제시한 목표주가는 이전에 교보증권이 제시한 목표주가 10만4000원 대비 15% 높은 수준이다.교보증권 박성국 애널리스트가 작성한 에스엠 리포트 주요내용은 아래와 같다.▶ EXO 백현과 샤이니 태민 전역으로 활동 라인업 최대▶ NCT DREAM과 에스파 성장 지속▶ 4Q22 Preview▶ FY23 Preview: 영업이익 YoY +60.4% 예상, Top-picks로 새롭게 제시권희진 '

## 뉴스핌 → 별도의 전처리 사항 없음

In [76]:
# 예시(전처리 전)
writer = '뉴스핌'  # 언론사 설정 
filtered_df[filtered_df['WRITER']== writer]["PARSED CONTENT"].iloc[0]  # 확인

'[서울=뉴스핌] 이지은 기자 = 한류로 시작했던 K팝의 시장이 전 세계로 이어지고 있다. 그룹 방탄소년단과 블랙핑크, 그리고 4세대 그룹이 해외에서 명성을 떨치면서 지난해 음반 수출액 또한 사상 최고치를 경신한 가운데 중동지역에서 급물살을 타고 있는 K팝 인기가 한몫을 한 것으로 나타났다.\n◆ \'빅3\' 시장에 쏠린 시장…중동 지역 치고 올라오나\n그룹 방탄소년단과 블랙핑크, 스트레이키즈 등이 미국 빌보드 차트에서 호성적을 거두면서 음반 수출액 역시 지난해 사상 최고치를 경신했다. 지난해 한국 음반을 가장 많이 수입해 간 국가는 일본으로 8574만9000달러(약 1065억원)을 기록했다. 이어 중국이 5132만6000달러(약 637억원), 미국 3887만7000달러(약 483억원) 등이 뒤따랐다.\n음반 수출 대상국 톱10에는 대만, 네덜란드, 태국, 홍콩, 독일, 인도네시아, 프랑스가 이름을 올렸다. K팝이 전 세계적으로 인기를 끌고 있지만, 음반 수출 지역은 여전히 일본, 중국, 미국 \'빅3\'에 쏠려 있는 상황이다.\n세 국가의 비중은 2021년 71.7%에서 지난해 75.5%로 3.8%P 상승했다. 특히 중국의 경우 \'한한령(한류 제한령)\'으로 K팝 스타들의 현지 공연이 막혔지만 수출액은 꾸준히 늘고 있는 추세이다.\n특히, 중동에서도 K팝 인기가 급물살을 타고 있다. 한국국제교류재단은 2021년 12월 기준으로 세계 한류 팬 1억5660만명 소위 \'메나(MENA)\'로 불리는 북아프리카·중동 지역 한류 팬이 10년 사이 130배 폭증한 것으로 집계했다.\n실제 지난해 9월 30일 사우디아라비아 수도 리야드 시티에서 열린 CJ ENM의 \'KCON\'에서는 4일간 2만명의 관객이 운집됐다. 해당 콘서트가 중동 지역에서 열린 것은 2016년 8000석 규모의 아랍에미리트(UAE) 공연 이후 두 번째이다.\n사우디에서 K팝의 인기가 거세지면서 그룹 블랙핑크도 오는 20일 사우디아라비아 리야드에서 콘서트를 개최한다. 또 한류스타의 배출지로 불린 

- 삭제할 내용 없음

## 조선경제

In [77]:
# 예시(전처리 전)
writer = '조선경제'  # 언론사 설정 
filtered_df[filtered_df['WRITER']== writer]["PARSED CONTENT"].iloc[0]  # 확인

'YG엔터테인먼트(와이지엔터테인먼트) 수장 양현석 총괄 프로듀서(PD)가 3년여 만에 공식 복귀했다. 블랙핑크 지수, 빅뱅 지드래곤 등 대형 아티스트가 활동을 예고했고, 베일에 감춰져 있던 신인 걸그룹도 공개됐다. YG는 그간 부실한 라인업과 높은 블랙핑크 의존도가 문제로 지적돼 왔는데, 라인업 강화와 함께 실적이 한층 개선될 것으로 전망된다.6일 관련 업계와 법조계에 따르면 ‘버닝썬 사건’과 전 소속 가수 비아이(본명 김한빈) 마약 수사 무마 의혹 등으로 2019년 대표 직함을 내려놓은 양 PD가 최근 법원으로부터 무죄 판결을 받았다. 마약 수사 무마 의혹과 관련해, 양 PD는 비아이의 마약 구매 사실을 경찰에 진술한 A씨를 협박한 혐의를 받았는데 1심 재판부는 양 PD가 A씨를 압박한 것은 사실이지만 A씨의 진술에 신빙성이 없다고 봤다.앞서 양 PD는 버닝썬 사건 관련 성매매 알선 혐의에 대해서도 증거 불충분을 이유로 무혐의 판결이 내려진 바 있다. 다만 불법 원정도박 혐의는 인정돼 벌금형을 선고받았다.\\n\\n잇단 논란으로 일선에서 물러났던 양 PD는 지난달 말 마약 수사 무마 의혹 1심 판결이 나온 뒤 열흘 만인 새해 첫 날, YG 공식 유튜브 채널에 공개된 영상을 통해 모습을 드러냈다. 양 PD는 “신사옥에서 처음으로 발표하게 되는 그룹”이라며 7인조 신인 걸그룹 ‘베이비몬스터’를 소개했다. 그는 “지난 4년간 월말평가를 통해 저를 놀래켰던 것처럼 세상을 놀래킬 스타가 됐으면 좋겠다는 희망사항이 있다”고 말했다. 양 PD가 아티스트 활동과 관련해 공식적으로 발언을 한 건 3년 6개월 만이다. 양 PD는 향후 신인 발굴과 육성에 전념할 계획인 것으로 알려졌다.베이비몬스터 데뷔 예고와 동시에 대형 아티스트 컴백도 줄줄이 예고됐다. 블랙핑크 멤버 지수가 제니, 로제, 리사에 이어 솔로 데뷔에 나선다. YG는 “지수가 현재 솔로 앨범 녹음에 박차를 가하고 있다. 월드투어 일정을 진행하고 있는 가운데 음반 재킷 사진 촬영까지 끝마치고 틈틈이 음악 작업에 매진

In [78]:
# 불필요한 내용 삭제
pattern = r'@chosun.*$'
filtered_df.loc[filtered_df['WRITER'] == writer, 'PARSED CONTENT'] = \
    filtered_df.loc[filtered_df['WRITER'] == writer, 'PARSED CONTENT'].apply(lambda text: re.sub(pattern, '', text) if isinstance(text, str) else text)

In [79]:
# 예시(전처리 후)
filtered_df[filtered_df['WRITER']== writer]["PARSED CONTENT"].iloc[0]

'YG엔터테인먼트(와이지엔터테인먼트) 수장 양현석 총괄 프로듀서(PD)가 3년여 만에 공식 복귀했다. 블랙핑크 지수, 빅뱅 지드래곤 등 대형 아티스트가 활동을 예고했고, 베일에 감춰져 있던 신인 걸그룹도 공개됐다. YG는 그간 부실한 라인업과 높은 블랙핑크 의존도가 문제로 지적돼 왔는데, 라인업 강화와 함께 실적이 한층 개선될 것으로 전망된다.6일 관련 업계와 법조계에 따르면 ‘버닝썬 사건’과 전 소속 가수 비아이(본명 김한빈) 마약 수사 무마 의혹 등으로 2019년 대표 직함을 내려놓은 양 PD가 최근 법원으로부터 무죄 판결을 받았다. 마약 수사 무마 의혹과 관련해, 양 PD는 비아이의 마약 구매 사실을 경찰에 진술한 A씨를 협박한 혐의를 받았는데 1심 재판부는 양 PD가 A씨를 압박한 것은 사실이지만 A씨의 진술에 신빙성이 없다고 봤다.앞서 양 PD는 버닝썬 사건 관련 성매매 알선 혐의에 대해서도 증거 불충분을 이유로 무혐의 판결이 내려진 바 있다. 다만 불법 원정도박 혐의는 인정돼 벌금형을 선고받았다.\\n\\n잇단 논란으로 일선에서 물러났던 양 PD는 지난달 말 마약 수사 무마 의혹 1심 판결이 나온 뒤 열흘 만인 새해 첫 날, YG 공식 유튜브 채널에 공개된 영상을 통해 모습을 드러냈다. 양 PD는 “신사옥에서 처음으로 발표하게 되는 그룹”이라며 7인조 신인 걸그룹 ‘베이비몬스터’를 소개했다. 그는 “지난 4년간 월말평가를 통해 저를 놀래켰던 것처럼 세상을 놀래킬 스타가 됐으면 좋겠다는 희망사항이 있다”고 말했다. 양 PD가 아티스트 활동과 관련해 공식적으로 발언을 한 건 3년 6개월 만이다. 양 PD는 향후 신인 발굴과 육성에 전념할 계획인 것으로 알려졌다.베이비몬스터 데뷔 예고와 동시에 대형 아티스트 컴백도 줄줄이 예고됐다. 블랙핑크 멤버 지수가 제니, 로제, 리사에 이어 솔로 데뷔에 나선다. YG는 “지수가 현재 솔로 앨범 녹음에 박차를 가하고 있다. 월드투어 일정을 진행하고 있는 가운데 음반 재킷 사진 촬영까지 끝마치고 틈틈이 음악 작업에 매진

## 한경뉴스

In [80]:
# 예시(전처리 전)
writer = '한경뉴스'  # 언론사 설정 
filtered_df[filtered_df['WRITER']== writer]["PARSED CONTENT"].iloc[1]  # 확인

'\\n\n\\n[ 최세영 기자 ] 국내 대형 연예기획사의 올해 콘서트 관객 수가 코로나19 이전보다 대폭 증가할 것이란 분석이 나왔다. 조정 과정을 거치던 대형 엔터주 주가도 실적 기대에 힘입어 최근 반등했다.\n9일 교보증권에 따르면 올해 국내 대형 엔터 4사(하이브, JYP엔터테인먼트, 에스엠, YG엔터테인먼트)의 콘서트 관객 수는 총 723만 명에 육박할 전망이다. 지난해 추정치(418만 명) 대비 72.9% 늘어난 수치다. 코로나19 이전인 2019년 관객 수(533만 명)와 비교해도 35.6% 많다.\n에스엠의 콘서트 관객 수가 가장 가파르게 늘어날 것으로 기대되고 있다. 이 회사의 올해 콘서트 예상 관객 수는 229만 명으로 전년보다 두 배 이상 증가할 전망이다. 전년 대비 영업이익 증가율도 엔터 4사 가운데 가장 높은 60.4%를 기록할 것이란 관측이다.\n박성국 교보증권 연구원은 “매출과 수익성에 크게 기여할 수 있는 돔 공연이 올해 다수 계획돼 있다”며 “올해 활동 예정인 국내 아티스트 중 블랙핑크, 트와이스, 슈퍼주니어 등 11개 그룹의 돔 투어가 가능하다”고 했다.\n증권가에선 콘서트 본격 재개 등의 영향으로 올해 대형 엔터 기업 실적이 뛸 것으로 보고 있다. 금융정보업체 에프앤가이드에 따르면 국내 대형 엔터 4사의 올해 합산 매출과 영업이익 예상치는 각각 3조4762억원, 5247억원이다. 전년(3조2417억원, 4837억원)보다 늘어난 수치다.\n지난달 말 하락하던 주가도 최근 다시 올랐다. 하이브 주가는 이날 3.27% 상승했다. 한 주간(1월 2~9일) 수익률은 2.4%다. 에스엠과 YG엔터 주가도 한 주간 각각 1.5%, 1.6% 올랐다. JYP엔터 주가는 지난 3거래일 연속 떨어지다가 이날 4% 가까이 뛰었다.\n최세영 기자\nⓒ 한국경제 & \n, 무단전재 및 재배포 금지\\n                                                                                     

In [81]:
# 불필요한 내용 삭제 1
# ⓒ 이후내용 모두삭제
pattern = r'ⓒ.*'
filtered_df.loc[filtered_df['WRITER'] == writer, 'PARSED CONTENT'] = \
    filtered_df.loc[filtered_df['WRITER'] == writer, 'PARSED CONTENT'].apply(lambda text: re.sub(pattern, '', str(text), flags=re.DOTALL | re.IGNORECASE))


In [82]:
# 불필요한 내용 삭제 2
# 기자 ] 이전 내용 모두삭제
pattern = r'.*기자\s?]'
filtered_df.loc[filtered_df['WRITER'] == writer, 'PARSED CONTENT'] = \
    filtered_df.loc[filtered_df['WRITER'] == writer, 'PARSED CONTENT'].apply(lambda text: re.sub(pattern, '', str(text), flags=re.DOTALL | re.IGNORECASE))


In [83]:
# 예시(전처리 후)
filtered_df[filtered_df['WRITER']== writer]["PARSED CONTENT"].iloc[1]

' 국내 대형 연예기획사의 올해 콘서트 관객 수가 코로나19 이전보다 대폭 증가할 것이란 분석이 나왔다. 조정 과정을 거치던 대형 엔터주 주가도 실적 기대에 힘입어 최근 반등했다.\n9일 교보증권에 따르면 올해 국내 대형 엔터 4사(하이브, JYP엔터테인먼트, 에스엠, YG엔터테인먼트)의 콘서트 관객 수는 총 723만 명에 육박할 전망이다. 지난해 추정치(418만 명) 대비 72.9% 늘어난 수치다. 코로나19 이전인 2019년 관객 수(533만 명)와 비교해도 35.6% 많다.\n에스엠의 콘서트 관객 수가 가장 가파르게 늘어날 것으로 기대되고 있다. 이 회사의 올해 콘서트 예상 관객 수는 229만 명으로 전년보다 두 배 이상 증가할 전망이다. 전년 대비 영업이익 증가율도 엔터 4사 가운데 가장 높은 60.4%를 기록할 것이란 관측이다.\n박성국 교보증권 연구원은 “매출과 수익성에 크게 기여할 수 있는 돔 공연이 올해 다수 계획돼 있다”며 “올해 활동 예정인 국내 아티스트 중 블랙핑크, 트와이스, 슈퍼주니어 등 11개 그룹의 돔 투어가 가능하다”고 했다.\n증권가에선 콘서트 본격 재개 등의 영향으로 올해 대형 엔터 기업 실적이 뛸 것으로 보고 있다. 금융정보업체 에프앤가이드에 따르면 국내 대형 엔터 4사의 올해 합산 매출과 영업이익 예상치는 각각 3조4762억원, 5247억원이다. 전년(3조2417억원, 4837억원)보다 늘어난 수치다.\n지난달 말 하락하던 주가도 최근 다시 올랐다. 하이브 주가는 이날 3.27% 상승했다. 한 주간(1월 2~9일) 수익률은 2.4%다. 에스엠과 YG엔터 주가도 한 주간 각각 1.5%, 1.6% 올랐다. JYP엔터 주가는 지난 3거래일 연속 떨어지다가 이날 4% 가까이 뛰었다.\n최세영 기자\n'

## 서울경제 → 별도의 전처리 사항 없음

## 파이낸셜 포인트

In [84]:
# 예시(전처리 전)
writer = '파이낸셜 포인트'  # 언론사 설정 
filtered_df[filtered_df['WRITER']== writer]["PARSED CONTENT"].iloc[0]  # 확인

'교보증권에서 9일 와이지엔터테인먼트(122870)에 대해 "연초부터 열일 중"라며 투자의견을 \'BUY\'로 제시하였고, 아울러 목표주가로는 64,000원을 내놓았다. 교보증권 박성국 애널리스트가 동종목에 대하여 이번에 제시한 \'BUY\'의견은 교보증권의 직전 매매의견에서 큰 변화없이 그대로 유지되는 것이고  전년도말의 매매의견을 기준으로  볼때도 역시 \'매수\'의견이 계속 유지되고 있는 상태이다. 그리고 최근 분기내 발표된 전체 증권사 리포트의 컨센서스와 비교를 해볼 경우에 오늘 발표된 투자의견은 전체의견에 수렴하고 있어, 이번 의견은 시장의 평균적인 기대감이 객관적으로 표현된 것으로 분석되고 있다. 또한 전일 종가 기준으로 볼때 동종목의 현주가는 이번에 제시된 목표가 대비 29%의 저평가 요인이 존재한다는 해석이 제시되고 있다. \n\n◆ Report briefing교보증권에서 와이지엔터테인먼트(122870)에 대해 "23년 1월 신인 걸그룹 \'베이비몬스터\' 론칭 공식화, 지수 솔로 앨범 발매, GD 컴백 예고 통해 아이콘 계약해지 및 위너 입대로 우려되던 One IP 리스크 해소. 빅뱅은 K-POP 아티스트 가운데 중국 QQ뮤직 팔로워 수 1위(1,392만)로 2위 블랙핑크의 두 배 이상. GD 실물앨범 발매시 22년 2분기 \'봄여름가을겨울\' 음원 정산금액 이상 수익창출 기대."라고 분석했다.  또한 교보증권에서 "22년 4분기 매출액 1,320억원(YoY +48.4%), 영업이익 222억원(YoY +67.6%)으로 컨센서스 대비 매출 하회, 영업이익 부합 전망. 23년 매출액 4,823억원(YoY +21.1%), 영업이익 647억원(YoY +21.8%) 전망."라고 밝혔다. \n◆ Report statistics교보증권의 동종목에 대한 최근 3개월 동안의 투자의견은 전체적으로 큰 변화없이 유지되고 있다. \n\n오늘 교보증권에서 발표된 \'BUY\'의견 및 목표주가 64,000원은 전체의견에 수렴하면서 시장의 평균적인 기대감이 비교적 객관적

In [85]:
# 불필요한 내용 삭제 1
# 투자의견 추이] 이후 내용 삭제

pattern = r'투자의견 추이\].*'
filtered_df.loc[filtered_df['WRITER'] == writer, 'PARSED CONTENT'] = \
    filtered_df.loc[filtered_df['WRITER'] == writer, 'PARSED CONTENT'].apply(lambda text: re.sub(pattern, '', str(text), flags=re.DOTALL | re.IGNORECASE))


In [86]:
# 불필요한 내용 삭제 2
# [전체 증권사 최근 리포트] 이후 내용 삭제
pattern = r'\[전체 증권사 최근 리포트\].*'
filtered_df.loc[filtered_df['WRITER'] == writer, 'PARSED CONTENT'] = \
    filtered_df.loc[filtered_df['WRITER'] == writer, 'PARSED CONTENT'].apply(lambda text: re.sub(pattern, '', str(text), flags=re.DOTALL | re.IGNORECASE))


In [87]:
# 예시(전처리 후)
filtered_df[filtered_df['WRITER']== writer]["PARSED CONTENT"].iloc[0]

'교보증권에서 9일 와이지엔터테인먼트(122870)에 대해 "연초부터 열일 중"라며 투자의견을 \'BUY\'로 제시하였고, 아울러 목표주가로는 64,000원을 내놓았다. 교보증권 박성국 애널리스트가 동종목에 대하여 이번에 제시한 \'BUY\'의견은 교보증권의 직전 매매의견에서 큰 변화없이 그대로 유지되는 것이고  전년도말의 매매의견을 기준으로  볼때도 역시 \'매수\'의견이 계속 유지되고 있는 상태이다. 그리고 최근 분기내 발표된 전체 증권사 리포트의 컨센서스와 비교를 해볼 경우에 오늘 발표된 투자의견은 전체의견에 수렴하고 있어, 이번 의견은 시장의 평균적인 기대감이 객관적으로 표현된 것으로 분석되고 있다. 또한 전일 종가 기준으로 볼때 동종목의 현주가는 이번에 제시된 목표가 대비 29%의 저평가 요인이 존재한다는 해석이 제시되고 있다. \n\n◆ Report briefing교보증권에서 와이지엔터테인먼트(122870)에 대해 "23년 1월 신인 걸그룹 \'베이비몬스터\' 론칭 공식화, 지수 솔로 앨범 발매, GD 컴백 예고 통해 아이콘 계약해지 및 위너 입대로 우려되던 One IP 리스크 해소. 빅뱅은 K-POP 아티스트 가운데 중국 QQ뮤직 팔로워 수 1위(1,392만)로 2위 블랙핑크의 두 배 이상. GD 실물앨범 발매시 22년 2분기 \'봄여름가을겨울\' 음원 정산금액 이상 수익창출 기대."라고 분석했다.  또한 교보증권에서 "22년 4분기 매출액 1,320억원(YoY +48.4%), 영업이익 222억원(YoY +67.6%)으로 컨센서스 대비 매출 하회, 영업이익 부합 전망. 23년 매출액 4,823억원(YoY +21.1%), 영업이익 647억원(YoY +21.8%) 전망."라고 밝혔다. \n◆ Report statistics교보증권의 동종목에 대한 최근 3개월 동안의 투자의견은 전체적으로 큰 변화없이 유지되고 있다. \n\n오늘 교보증권에서 발표된 \'BUY\'의견 및 목표주가 64,000원은 전체의견에 수렴하면서 시장의 평균적인 기대감이 비교적 객관적

## NSP, NSP통신

In [88]:
# 예시(전처리 전)
writer = ['NSP','NSP통신']  # 언론사 설정 
filtered_df[filtered_df['WRITER'].isin(writer)]["PARSED CONTENT"].iloc[0]  # 확인

'\n(서울=NSP통신) 류수운 기자 = 그룹 엔시티 드림(NCT DREAM)이 써클차트(구. 가온차트) 리테일(소매점) 월간 앨범 순위에서 정상에 올랐다.\n_x000D_\n\n_x000D_\n써클차트가 2일 공개한 2022년 12월 리테일 앨범차트에 따르면 엔시티 드림의 겨울 스페셜 미니앨범 ‘Candy’(캔디)가 집계 기준이 되는 한 달 동안 가장 많은 81만4336장의 판매량으로 1위를 차지했다.\n_x000D_\n\n_x000D_\n앨범 ‘Candy’는 지난 1996년 그룹 에이치오티(H.O.T.)가 발표한 ‘Candy’를 엔시티 드림만의 색깔로 재해석한 동명의 타이틀 곡을 포함해 총 6곡으로 구성돼 있다.\n\n\n\n2~5위는 ▲있지(ITZY) ‘CHESHIRE’(체셔) (33만5686장) ▲레드벨벳(Red Velvet) ‘The ReVe Festival 2022 - Birthday’(더 리브 페스티벌 2022 - 버스데이) (23만109장) ▲알엠(RM) ‘Indigo’(인디고) (21만8388장) ▲알엠 ‘Indigo (Weverse)’ (16만6913장) 순으로 랭크됐다.\n_x000D_\n\n_x000D_\n한편 이날 함께 발표된 53주차(12.25~31) 리테일 앨범차트 1위 역시 엔시티 드림 ‘Candy’가 차지하면서 2주 연속 정상을 지켰다. 집계 기간 중 구랍 27일 오후 5시에 가장 많은 판매량을 나타냈다.\n_x000D_\n\n_x000D_\n이 기간 일간 리테일 앨범차트에서는 △25, 27일 엔시티 드림 ‘Candy’ △26, 29, 31일 에스엠타운(SMTOWN) ‘2022 Winter SMTOWN : SMCU PALACE’(2022 윈터 에스엠타운 : SMCU 팰리스) △28일 웨이션브이(WayV) ‘Phantom’(팬텀) △30일 에이티즈(ATEEZ) ‘SPIN OFF : FROM THE WITNESS’(스핀 오프 : 프롬 더 위트니스)가 각각 정상에 올랐다.\n\n\n\nNSP통신 류수운 기자 swryu64@nspna.c

In [89]:
# 불필요한 내용 삭제 1
filtered_df.loc[filtered_df['WRITER'].isin(writer), 'PARSED CONTENT'] = \
    filtered_df.loc[filtered_df['WRITER'].isin(writer), 'PARSED CONTENT'].str.replace("ⓒ 한국의 경제뉴스통신사 NSP통신·NSP TV. 무단전재-재배포 금지.", "", regex=False)

In [90]:
# 불필요한 내용 삭제 2
# 기자 =이전 내용 모두 삭제
pattern = r'.*기자\s?='
filtered_df.loc[filtered_df['WRITER'].isin(writer), 'PARSED CONTENT'] = \
    filtered_df.loc[filtered_df['WRITER'].isin(writer), 'PARSED CONTENT'].apply(lambda text: re.sub(pattern, '', str(text), flags=re.DOTALL | re.IGNORECASE))


In [91]:
# 예시(전처리 후)
filtered_df[filtered_df['WRITER'].isin(writer)]["PARSED CONTENT"].iloc[0]

' 그룹 엔시티 드림(NCT DREAM)이 써클차트(구. 가온차트) 리테일(소매점) 월간 앨범 순위에서 정상에 올랐다.\n_x000D_\n\n_x000D_\n써클차트가 2일 공개한 2022년 12월 리테일 앨범차트에 따르면 엔시티 드림의 겨울 스페셜 미니앨범 ‘Candy’(캔디)가 집계 기준이 되는 한 달 동안 가장 많은 81만4336장의 판매량으로 1위를 차지했다.\n_x000D_\n\n_x000D_\n앨범 ‘Candy’는 지난 1996년 그룹 에이치오티(H.O.T.)가 발표한 ‘Candy’를 엔시티 드림만의 색깔로 재해석한 동명의 타이틀 곡을 포함해 총 6곡으로 구성돼 있다.\n\n\n\n2~5위는 ▲있지(ITZY) ‘CHESHIRE’(체셔) (33만5686장) ▲레드벨벳(Red Velvet) ‘The ReVe Festival 2022 - Birthday’(더 리브 페스티벌 2022 - 버스데이) (23만109장) ▲알엠(RM) ‘Indigo’(인디고) (21만8388장) ▲알엠 ‘Indigo (Weverse)’ (16만6913장) 순으로 랭크됐다.\n_x000D_\n\n_x000D_\n한편 이날 함께 발표된 53주차(12.25~31) 리테일 앨범차트 1위 역시 엔시티 드림 ‘Candy’가 차지하면서 2주 연속 정상을 지켰다. 집계 기간 중 구랍 27일 오후 5시에 가장 많은 판매량을 나타냈다.\n_x000D_\n\n_x000D_\n이 기간 일간 리테일 앨범차트에서는 △25, 27일 엔시티 드림 ‘Candy’ △26, 29, 31일 에스엠타운(SMTOWN) ‘2022 Winter SMTOWN : SMCU PALACE’(2022 윈터 에스엠타운 : SMCU 팰리스) △28일 웨이션브이(WayV) ‘Phantom’(팬텀) △30일 에이티즈(ATEEZ) ‘SPIN OFF : FROM THE WITNESS’(스핀 오프 : 프롬 더 위트니스)가 각각 정상에 올랐다.\n\n\n\nNSP통신 류수운 기자 swryu64@nspna.com\n_x000D_\n저작권자    

##  이투데이

In [92]:
# 예시(전처리 전)
writer = '이투데이'  # 언론사 설정 
filtered_df[filtered_df['WRITER']== writer]["PARSED CONTENT"].iloc[0]  # 확인

'[이투데이/손민지 기자()]\n\n메리츠증권이 하이브에 대해 올해 IP 다각화가 성공해 호실적을 기록할 것으로 전망했다. 투자의견은 ‘매수’ 유지, 목표주가는 기존 17만 원에서 21만 원으로 상향조정했다. 전 거래일 기준 종가는 18만8000원이다.\n\n20일 정지수 메리츠증권 연구원은 “하이브의 2022년 4분기 연결 실적은 매출액은 4342억 원, 영업이익은 509억 원으로 컨센서스(시장 전망치)를 하회할 전망”이라며 “엔하이픈, TXT의 콘서트 모객 규모가 작았고, 10월 BTS의 부산 무료 콘서트 개최 등으로 공연 관련 수익성이 대체로 부진했다”고 분석했다.\n\n이어 “지난 12월 일본에서 데뷔한 신인 그룹(&TEAM) 관련 제작원가 증가도 4분기 수익성 부진의 원인으로 파악된다”고 덧붙였다.\n\n다만 정 연구원은 “2023년 1분기에는 1월 TXT, 3월 세븐틴 유닛 앨범 등 총 3~4개의 앨범 실적이 반영되며, 엔하이픈과 TXT의 월드투어 실적은 대부분은 2023년 2분기에 인식될 전망”이라며 “엔하이픈과 TXT는 2023년 콘서트 모객 규모를 2배 가까이 확대함으로써 외형과 이익이 모두 성장할 전망”이라고 했다.\n\n또 그는 “위버스는 2023년 2분기부터 구독 모델 도입을 도입할 예정”이라며 “2023년 연결 매출액과 영업이익은 각각 1조8144억 원, 2273억 원을 전망한다”고 봤다.\n\n이어 “뉴진스 ‘Ditto’(디토)는 최근 빌보드 ‘핫 100’ 차트에 진입하며 4세대 K팝 아티스트 중 가장 빠른 성장세를 보인다”며 “2023년에는 BTS 공백에도 불구하고 기존 지식재산권(IP)에 신규 IP 3팀이 추가될 예정”이라고 덧붙였다.[종목기사]    ●     ●     ● [오늘의 핫클릭]    ●    ●    ●    ●    ●    ●    ●  이투데이(www.etoday.co.kr), 무단전재 및 수집, 재배포금지 '

In [93]:
# 불필요한 내용 삭제 1
#기자()]이전내용 모두 삭제
pattern = r'.*기자\s?\(\)]'
filtered_df.loc[filtered_df['WRITER']==writer, 'PARSED CONTENT'] = \
    filtered_df.loc[filtered_df['WRITER']==writer, 'PARSED CONTENT'].apply(lambda text: re.sub(pattern, '', str(text), flags=re.DOTALL | re.IGNORECASE))


In [94]:
# 불필요한 내용 삭제 2
# [오늘의 핫클릭] 이후내용 모두 삭제
pattern = r'\[오늘의 핫클릭\].*$'
filtered_df.loc[filtered_df['WRITER']==writer, 'PARSED CONTENT'] = \
    filtered_df.loc[filtered_df['WRITER']==writer, 'PARSED CONTENT'].apply(lambda text: re.sub(pattern, '', str(text), flags=re.DOTALL | re.IGNORECASE))


In [95]:
# 불필요한 내용 삭제 3
# [종목기사] 이후내용 모두 삭제
pattern = r'\[종목기사\].*$'
filtered_df.loc[filtered_df['WRITER']==writer, 'PARSED CONTENT'] = \
    filtered_df.loc[filtered_df['WRITER']==writer,  'PARSED CONTENT'].apply(lambda text: re.sub(pattern, '', str(text), flags=re.DOTALL | re.IGNORECASE))

In [96]:
# 예시(전처리 후)
filtered_df[filtered_df['WRITER']== writer]["PARSED CONTENT"].iloc[0]

'\n\n메리츠증권이 하이브에 대해 올해 IP 다각화가 성공해 호실적을 기록할 것으로 전망했다. 투자의견은 ‘매수’ 유지, 목표주가는 기존 17만 원에서 21만 원으로 상향조정했다. 전 거래일 기준 종가는 18만8000원이다.\n\n20일 정지수 메리츠증권 연구원은 “하이브의 2022년 4분기 연결 실적은 매출액은 4342억 원, 영업이익은 509억 원으로 컨센서스(시장 전망치)를 하회할 전망”이라며 “엔하이픈, TXT의 콘서트 모객 규모가 작았고, 10월 BTS의 부산 무료 콘서트 개최 등으로 공연 관련 수익성이 대체로 부진했다”고 분석했다.\n\n이어 “지난 12월 일본에서 데뷔한 신인 그룹(&TEAM) 관련 제작원가 증가도 4분기 수익성 부진의 원인으로 파악된다”고 덧붙였다.\n\n다만 정 연구원은 “2023년 1분기에는 1월 TXT, 3월 세븐틴 유닛 앨범 등 총 3~4개의 앨범 실적이 반영되며, 엔하이픈과 TXT의 월드투어 실적은 대부분은 2023년 2분기에 인식될 전망”이라며 “엔하이픈과 TXT는 2023년 콘서트 모객 규모를 2배 가까이 확대함으로써 외형과 이익이 모두 성장할 전망”이라고 했다.\n\n또 그는 “위버스는 2023년 2분기부터 구독 모델 도입을 도입할 예정”이라며 “2023년 연결 매출액과 영업이익은 각각 1조8144억 원, 2273억 원을 전망한다”고 봤다.\n\n이어 “뉴진스 ‘Ditto’(디토)는 최근 빌보드 ‘핫 100’ 차트에 진입하며 4세대 K팝 아티스트 중 가장 빠른 성장세를 보인다”며 “2023년에는 BTS 공백에도 불구하고 기존 지식재산권(IP)에 신규 IP 3팀이 추가될 예정”이라고 덧붙였다.'

## 파이낸셜뉴스

In [97]:
# 예시(전처리 전)
writer = '파이낸셜뉴스'  # 언론사 설정 
filtered_df[filtered_df['WRITER']== writer]["PARSED CONTENT"].iloc[0]  # 확인

"그룹 몬스타엑스(MONSTA X)와 CIX(씨아이엑스)가 한터차트 1월 3주 주간차트 정상에 등극했다.\n세계 유일의 실시간 음악차트인 한터차트는 오늘(16일) 오전 10시, 2022년 1월 3주 차 주간 월드차트, 음반차트, 그리고 인증차트를 발표했다.\n음반, 음원, 인증, SNS, 미디어 등 케이팝 아티스트의 글로벌 데이터를 바탕으로 발표되는 월드차트는 종합차트의 의미를 갖는다. 몬스타엑스는 1월 3주 주간 월드차트에서 1만7087.71점의 월드 지수로 1위를 차지했다. 이어 주간 월드차트에서 2위는 방탄소년단, 3위는 NCT 드림이 차트에 올랐다.\n한터차트의 차트 기준인 '음반 지수'로 선정된 지난주 음반차트 1위는 37만2257.28점(주간 음반 판매량 32만6503장)을 기록한 몬스타엑스의 'REASON' 앨범이었다. 몬스타엑스는 컴백과 동시에 월드차트에 이어 음반차트까지 점령했다.\n이어 2위는 SF9의 'THE PIECE OF9', 3위는 문빈&산하의 'INCENSE', 4위는 웨이션브이의 'Phantom', 5위는 슈퍼주니어의 'The Road' 앨범이 차례로 차트인 했다. \n6위는 뉴진스의 'OMG', 7위는 레드벨벳의 'The ReVe Festival 2022 - Birthday', 8위는 뉴진스의 'New Jeans', 9위는 엔시티 드림의 'Candy', 10위는 르세라핌의 'ANTIFRAGILE' 앨범 순이었다. \n주간 인증차트 1위는 CIX의 ''OK' Episode 1 : OK Not' 앨범이었다. CIX는 5개월 전에 발매했던 해당 앨범으로 1위에 오르며 글로벌 인기를 입증했다.\n이어서 2위는 NCT 드림의 'Candy', 3위는 이펙스의 '사랑의 서 Chapter 1. Puppy Love', 4위는 문빈&산하의 'INCENSE', 5위는 스트레이 키즈의 'MAXIDENT', 6위는 RM의 'Indigo', 7위는 세븐틴의 'Face the Sun', 8위는 원호의 'Bittersweet', 9위는 웨이션브이의 'Phan

In [98]:
# 불필요한 내용 삭제 1
# 기자】 이전내용 모두 삭제
pattern = r'.*기자\s?】'
filtered_df.loc[filtered_df['WRITER']==writer, 'PARSED CONTENT'] = \
    filtered_df.loc[filtered_df['WRITER']==writer, 'PARSED CONTENT'].apply(lambda text: re.sub(pattern, '', str(text), flags=re.DOTALL | re.IGNORECASE))


In [99]:
# 불필요한 내용 삭제 2
# 저작권자 ⓒ 파이낸셜뉴스 이후내용 모두삭제

pattern = r'저작권자 ⓒ 파이낸셜뉴스.*'
filtered_df.loc[filtered_df['WRITER'] == writer, 'PARSED CONTENT'] = \
    filtered_df.loc[filtered_df['WRITER'] == writer, 'PARSED CONTENT'].apply(lambda text: re.sub(pattern, '', str(text), flags=re.DOTALL | re.IGNORECASE))


In [100]:
# 불필요한 내용 삭제 3
# [파이낸셜뉴스] 삭제
filtered_df.loc[filtered_df['WRITER'] == writer, 'PARSED CONTENT'] = \
    filtered_df.loc[filtered_df['WRITER'] == writer, 'PARSED CONTENT'].str.replace("[파이낸셜뉴스]", "", regex=False)

In [101]:
# 예시(전처리 후)
filtered_df[filtered_df['WRITER']== writer]["PARSED CONTENT"].iloc[10]

'\xa0올해 엔터주들의 전망이 밝다. 코로나19라는 긴 터널을 지나, 본격적으로 아티스트들이 국내외 팬들과 만날 기회가 많아져서다. 올해 신인그룹들도 대거 데뷔를 앞두고 있어 실적 개선 및 주가 상승에 대한 기대를 모으고 있다. \n\xa019일 한국거래소에 따르면 올들어 엔터테인먼트 종목들의 주가가 대부분 오름세다. \n\xa0큐브엔터의 경우 올들어서만 주가가 34.3% 뛰었다. 올해 영업이익이 두배 증가할 것이라는 전망과 (여자)아이들의 월드투어 성공이 기대돼서다. \n\xa0와이지엔터테인먼트도 20%, 하이브도 8.4%의 주가상승률을 나타냈다. 지난해 다 떨어지는 와중에서도 주가가 올랐던 에스엠(0.1%)과 JYP Ent(-1.3%)만 약보합세를 보이고 있다. \n\xa0지난해 한해동안 하이브(-50.5%), 큐브엔터(-34.7%), 와이지(-22.1%) 등은 주가가 큰폭으로 떨어진 반면 JYP Ent 주가는 33.7%, 에스엠의 경우 4.2% 오른 바 있다. \n\xa0증권사들은 올해 본격적으로 엔터주에 대한 비중 확대 의견을 제시하고 있다. \n\xa0박성국 교보증권 연구원은 "엔터 업종의 올해 첫 번째 키워드는 콘서트 온기 재개"라고 꼽으며 "엔터 4사(큐브엔터 제외)의 올해 합산 콘서트 관객수는 코로나 이전인 2019년 대비 35.6% 증가할 것"으로 예상했다. \n\xa0최근 중국의 한한령 완화 기조도 긍정적이다. 실제로 중국향 앨범수출금액은 최근 7년새 800.5% 증가하며 중국 내 K-POP 확산은 지속되고 있다는 판단이다. \n\xa0박 연구원은 "두 번째 키워드는 신인그룹 데뷔"라며 "엔터 4사의 올해 데뷔예정 신인그룹은 총 12개팀으로, 과거 어느 때보다 많다. 대형기획사 신인그룹의 앨범판매량이 증가하면서 수익화 시기가 앞당겨졌고, 신인그룹 론칭에 대한 리스크가 과거 대비 감소했기 때문"이라고 풀이했다. \n\xa0박수영 한화투자증권 연구원도 "올해 존재들의 성장에 주목해야 한다"며 괄목할만한 존재들의 성장 대표 사례로는 스트레이키즈(

## 머니투데이 방송

In [102]:
# 예시(전처리 전)
writer = '머니투데이방송'  # 언론사 설정 
filtered_df[filtered_df['WRITER']== writer]["PARSED CONTENT"].iloc[0]  # 확인

'NH투자증권은 17일 하이브에 대해 BTS 단체 활동 부재에도 강한 신인과 기대되는 신사업이 기대된다고 분석했다. 투자의견은 매수를, 목표주가는 25만원을 유지했다. 이화정 NH투자증권 연구원은 기업분석보고서를 통해 "하이브의 지난해 4분기 연결기준 실적은 매출액 4008억원, 영업이익 504억원을 기록해 시장 기대치를 소폭 하회할 것"이라 밝혔다. 그는 "지난 2021년 BTS LA투어와 달리 세븐틴 아시아 투어의 경우 도시 이동이 잦고, 회당 모객이 적어 수익성이 낮았다"라면서도 "진 솔로(100만장), RM 솔로(75만장), 르세라핌 미니(75만장) 및 세븐틴 일본(100만장)과 같은 신보 판매 호조에 주목해야 한다"고 설명했다. 이 연구원은 "BTS 순차적 군입대로 단체 활동 부재에도 BTS의 멤버별 솔로 활동이 공백을 메꾸고 있고 지난해 데뷔한 3팀(르세라핌, 뉴진스, &팀)이 모두 흥행에 성공해 BTS 외 라인업도 흥행 지속 중"이라고 평가했다. 이어 "뉴진스가 데뷔 반년만에 신곡 \'Ditto\'를 통해 빌보드 메인 싱글차트 Hot 100에 진입할 것으로 예상된다"며 "블랙핑크의 미국 내 대중 인지도가 불어나기 시작한 시발점이 2018년 \'뚜두뚜두\' Hot 100 진입이었음을 고려하면 눈여겨볼 만하다"고 덧붙였다. 플랫폼 부문에서는 "브이라이브 통합 완료로 외부 입점 아티스트 및 이용자 유의미하게 성장 중이고 지난해 12월 뉴진스 전용 플랫폼 \'포닝\'으로 테스트를 마친 월별 구독 서비스의 경우 올해 2분기 위버스 플랫폼 전체에도 적용될 전망"이라고 분석했다. 이상현 머니투데이방송 MTN 기자[머니투데이방송 이상현 기자]<저작권자 ⓒ \'부자되는 좋은습관 대한민국 경제채널 MTN\'>[MTN 인기기사][MTN 핫이슈기사] '

In [103]:
# 불필요한 내용 삭제 1
# 머니투데이방송 MTN 기자 이후내용 모두삭제
pattern = r'머니투데이방송 MTN 기자.*'
filtered_df.loc[filtered_df['WRITER'] == writer, 'PARSED CONTENT'] = \
    filtered_df.loc[filtered_df['WRITER'] == writer, 'PARSED CONTENT'].apply(lambda text: re.sub(pattern, '', str(text), flags=re.DOTALL | re.IGNORECASE))


In [104]:
# 예시(전처리 후)
filtered_df[filtered_df['WRITER']== writer]["PARSED CONTENT"].iloc[0]

'NH투자증권은 17일 하이브에 대해 BTS 단체 활동 부재에도 강한 신인과 기대되는 신사업이 기대된다고 분석했다. 투자의견은 매수를, 목표주가는 25만원을 유지했다. 이화정 NH투자증권 연구원은 기업분석보고서를 통해 "하이브의 지난해 4분기 연결기준 실적은 매출액 4008억원, 영업이익 504억원을 기록해 시장 기대치를 소폭 하회할 것"이라 밝혔다. 그는 "지난 2021년 BTS LA투어와 달리 세븐틴 아시아 투어의 경우 도시 이동이 잦고, 회당 모객이 적어 수익성이 낮았다"라면서도 "진 솔로(100만장), RM 솔로(75만장), 르세라핌 미니(75만장) 및 세븐틴 일본(100만장)과 같은 신보 판매 호조에 주목해야 한다"고 설명했다. 이 연구원은 "BTS 순차적 군입대로 단체 활동 부재에도 BTS의 멤버별 솔로 활동이 공백을 메꾸고 있고 지난해 데뷔한 3팀(르세라핌, 뉴진스, &팀)이 모두 흥행에 성공해 BTS 외 라인업도 흥행 지속 중"이라고 평가했다. 이어 "뉴진스가 데뷔 반년만에 신곡 \'Ditto\'를 통해 빌보드 메인 싱글차트 Hot 100에 진입할 것으로 예상된다"며 "블랙핑크의 미국 내 대중 인지도가 불어나기 시작한 시발점이 2018년 \'뚜두뚜두\' Hot 100 진입이었음을 고려하면 눈여겨볼 만하다"고 덧붙였다. 플랫폼 부문에서는 "브이라이브 통합 완료로 외부 입점 아티스트 및 이용자 유의미하게 성장 중이고 지난해 12월 뉴진스 전용 플랫폼 \'포닝\'으로 테스트를 마친 월별 구독 서비스의 경우 올해 2분기 위버스 플랫폼 전체에도 적용될 전망"이라고 분석했다. 이상현 '

# 공통 전처리

- 모든 언론사들에서 공통적으로 포함하고 있는 전처리 사항을 다음과 같이 처리함 

In [105]:
# _x000D_ 삭제
filtered_df['PARSED CONTENT'] = \
    filtered_df['PARSED CONTENT'].str.replace("_x000D_", "", regex=False)

In [106]:
# \xa0 삭제
filtered_df['PARSED CONTENT'] = \
    filtered_df['PARSED CONTENT'].str.replace("\xa0", "", regex=False)

In [107]:
# r_start 삭제
filtered_df['PARSED CONTENT'] = \
    filtered_df['PARSED CONTENT'].str.replace("r_start", "", regex=False)

In [108]:
# \n 삭제
filtered_df['PARSED CONTENT'] = \
    filtered_df['PARSED CONTENT'].str.replace("\n", "", regex=False)

In [109]:
# \r 삭제
filtered_df['PARSED CONTENT'] = \
    filtered_df['PARSED CONTENT'].str.replace("\r", "", regex=False)

In [110]:
# _ 삭제
filtered_df['PARSED CONTENT'] = \
    filtered_df['PARSED CONTENT'].str.replace("_", "", regex=False)

In [111]:
# \ 삭제
filtered_df['PARSED CONTENT'] = filtered_df['PARSED CONTENT'].str.replace("\\", "", regex=False)

In [112]:
# / 삭제
filtered_df['PARSED CONTENT'] = filtered_df['PARSED CONTENT'].str.replace("/", "", regex=False)

In [113]:
# 전처리 결과 확인 
filtered_df

,NEWS_YM,DATE_TIME,WRITER,SUBCATEGORY,TITLE,IMPORTANCE,ITEM_NAME,TAG_LIST,PARSED CONTENT
4,202301,20230102_084347,eDaily,사회,BTS 자리 넘본다…유망주는 스트레이 키즈-KB,67.91,NaN,BTS| 판매량| NCT| 넥스트| 엔하이픈| 블랙핑크| 에스파| 아이브,"- KB증권 “군 입대 BTS에 도전 이어질 것”- “스트레이 키즈, 군 이슈 없고..."
7,202301,20230104_082655,eDaily,경제,BTS 군입대에도 K팝 주가 선방 전망-IBK,67.81,NaN,판매량| 방탄소년단| BTS| 개별| 공연| 글로벌,"- 이환욱 “K팝 앨범 판매 증가, 하이브 주가 상향”[이데일리 최훈길 기자] 방탄..."
9,202301,20230102_100838,eDaily,기술,"[특징주]와이지엔터, 베이비몬스터 데뷔 예고에 6%대↑",68.31,와이지엔터테인먼트,블랙핑크,[이데일리 이정현 기자] 신인 걸그룹 공개를 예고한 와이지엔터테인먼트(122870)...
18,202301,20230119_063002,eDaily,경제,BTS·블랙핑크에 뉴진스까지..K-스타 인기 올라탄 명품,50.47,NaN,블랙핑크| 명품| BTS| 글로벌| 콘텐츠| 방탄소년단| 빅뱅,- 디올은 지민 발렌티노는 슈가 앰버서더 발탁- 뉴진스 하니·혜인도 구찌·루이비통과...
20,202301,20230116_132720,eDaily,기술,"자이언트스텝, '2022 앤어워드' 디지털 미디어·서비스 부문 장관상 수상",44.42,자이언트스텝,디지털| 미디어| 콘텐츠,"- 올해 3관왕…""콘텐츠 제작 역량 강화에 집중할 것""[이데일리 이용성 기자] 리얼..."
...,...,...,...,...,...,...,...,...,...
16156,202306,20230601_163720,한경뉴스,경제,NH아문디 'K-pop ETF' 수익률 43%…엔터 ETF 1위,10.99,NaN,미디어| 엔터테인먼트| 글로벌,NH아문디자산운용의 K팝 테마 상장지수펀드(ETF)인 'HANARO Fn K-POP...
16159,202306,20230605_185336,한경뉴스,사회,"K팝 아이돌의 정석…홀로선 태연, 1만8000명 모았다[리뷰]",21.48,NaN,공연| 소녀시대| 코로나19,10년 넘게 최고의 자리를 지키는 데는 그만한 이유가 있었다. 홀로 무대에 섰을 때...
16160,202306,20230606_174433,한경뉴스,기술,"다음 K테마는 무엇이냐…""엔터·식품·의료기기 주목""",12.57,하이브|JYP Ent.|에스엠|와이지엔터테인먼트|농심|덴티움|제이시스메디칼|엔씨소프...,식품| 의료기기| 게임| 미디어| 중국| 산업| 엔터테인먼트| 콘텐츠| 유튜브| 미...,올 들어 증시가 배터리와 반도체 등 일부 테마를 중심으로 순환매 조짐을 보이면서 ...
16166,202306,20230612_143036,한경뉴스,경제,"당일 증권사 관심 종목 - SK하이닉스 , 하이브 등",13.73,NaN,증권사| 환경,빅데이터를 기반으로 투자 이슈를 분석하는 한경라씨로에 따르면 06월 12일 2시 3...


# 3차 키워드 필터링
- 태그 제거하기 전 K-POP 엔터테인먼트 관련 키워드로 1차 필터링을 진행, HTML의 태그가 제거된 뉴스 본문에 대하여 2차 필터링을 진행하였음
- 언론사별로 전처리가 끝난 데이터에 대해 3차 필터링을 진행하여 관련성이 높은 기사만 추출하도록 함 
- 2가지 기준을 사용하여 추출함 
    - 1. PARSED CONTENT 변수를 기준으로 
    - 2. TITLE, ITEM_NAME, TAG_LIST 변수를 기준으로 

## (1) PARSED CONTENT변수를 기준으로

In [114]:
keywords = [
    '방시혁','하이브','에스엠','이수만','와이지','양현석','제이와이피','박진영',
    
    '뉴진스','방탄소년단','르세라핌',
    '세븐틴','투모로우바이투게더',
    '프로미스나인','나나','이현','미드낫','엔하이픈',
    '백호','강동호','황민현','예하나','성연','지코',
    '앤팀','히라테 유리나','보이넥스트도어',
    
    '블랙핑크','트레저','악뮤','악동뮤지션',
    '위너','지디','권지용',
    '젝스키스',
    
    '트와이스','엔믹스', '데이식스',
    'itzy','채령','류진','유나','예지'
    
    '스트레이키즈','투피엠','보이스토리',
    '야오천','니지유','엑스디너리 히어로즈',
    'boa',
    '에스파',
    '엑소',
    
    '샤이니','엔시티','엔시티 127','엔시티 드림',    
    '동방신기','슈퍼주니어','소녀시대','엑소'
    '레드벨벳','웨이션브이','슈퍼엠',
    '갓더비트','갓 더 비트','라이즈'

]

# 특정 단어 목록을 정규식 패턴으로 변환
keywords_pattern = '|'.join(keywords)

# 'PARSED CONTENT' 열에서 정규식 패턴을 검색하여 추출
filtered_df = filtered_df[filtered_df['PARSED CONTENT'].str.contains(keywords_pattern, flags=re.IGNORECASE, regex=True)]

# 특정 키워드가 3번 이상 포함된 행만 남기기
filtered_df = filtered_df[filtered_df['PARSED CONTENT'].apply(lambda text: sum(1 for keyword in keywords if keyword.lower() in text.lower()) >= 3)]


In [115]:
# 필터링 결과 확인
filtered_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3189 entries, 4 to 16174
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   NEWS_YM         3189 non-null   int64  
 1   DATE_TIME       3189 non-null   object 
 2   WRITER          3189 non-null   object 
 3   SUBCATEGORY     3189 non-null   object 
 4   TITLE           3189 non-null   object 
 5   IMPORTANCE      3189 non-null   float64
 6   ITEM_NAME       1654 non-null   object 
 7   TAG_LIST        2424 non-null   object 
 8   PARSED CONTENT  3189 non-null   object 
dtypes: float64(1), int64(1), object(7)
memory usage: 249.1+ KB


## TITLE, ITEM_NAME, TAG_LIST를 기준으로 
- 아래의 키워드를 ‘TITLE’ 에 포함, “또는” ‘ITEM_NAME’ 이나 ‘TAG_LIST’에 포함하는 행만 추출 

In [116]:
keywords = [
    '엔터','데뷔','아이돌','걸그룹','보이그룹','기획사','스트리밍','엔터주','빌보드','K팝','음반','음원차트','한류',
    '아티스트','콘서트','멜론','빅4','앨범','팬미팅','K-팝','MV','엔터업계','엔터사','엔터계','K-POP','K콘텐츠',
    
    'HYBE', 'Hybe','hybe','하이브 엔터','하이브','방시혁','위버스','민희진'
    '방탄소년단','BTS','지민','뷔', '지민','RM','제이홉','슈가','정국','전정국'
    '뉴진스','하니','민지','혜인','해린','다니엘'
    'TXT','투모로우바이투게더',
    '르세라핌','허윤진','사쿠라','은채','채원','카즈하',
    '세븐틴','지코','엔하이픈','부석순',
    '프로미스나인','나나','이현','미드낫',
    '백호','강동호','황민현','예하나','성연',
    '앤팀','히라테 유리나','보이넥스트도어',
    
    'SM','Sm','sm''SM엔터','SM C&C','에스엠','이수만','디어유'
    '보아',
    'NCT','엔시티',
    '엑소','찬열','세훈','수호','첸','시우민','디오','백현','카이'
    '에스파','윈터','카리나','닝닝','지젤',
    '샤이니','민호','키','태민','온유'
    '슈퍼주니어',
    '동방신기','유노윤호','최강창민'
    '소녀시대','윤아','태연','서현','티파니','소녀시대 수영','서현'
    '레드벨벳','아이린','슬기','예리','조이','웬디'
    '웨이션브이',
    '슈퍼엠',
    '갓더비트','갓 더 비트',
    '라이즈',
    
    'YG','와이지','와이지엔터','YG엔터','와이지엔터테인먼트','YG PLUS','양현석',
    '베이비몬스터','블랙핑크','리사','지드래곤','제니','트레저','악뮤','AKMU','악동뮤지션',
    '위너','지디','권지용','GD'
    '젝스키스',

    'JYP','박진영','JYP Ent.','트와이스','스트레이 키즈','스트레이키즈','스키즈','있지',
    '엔믹스', '데이식스','DAY6','itzy','채령','류진','유나','예지', '스트레이키즈',
    '투피엠','보이스토리','야오천','니지유','엑스디너리 히어로즈'
]

# 특정 단어 목록을 정규식 패턴으로 변환
keywords_pattern = '|'.join(keywords)

# 'TITLE', 'ITEM_NAME', 'TAG_LIST' 열에서 정규식 패턴을 검색하여 추출
filtered_df = filtered_df[
    (filtered_df['TITLE'].str.contains(keywords_pattern, flags=re.IGNORECASE, regex=True)) |
    (filtered_df['ITEM_NAME'].str.contains(keywords_pattern, flags=re.IGNORECASE, regex=True)) |
    (filtered_df['TAG_LIST'].str.contains(keywords_pattern, flags=re.IGNORECASE, regex=True))
]
  

In [117]:
# 필터링 결과 확인 
filtered_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2939 entries, 4 to 16166
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   NEWS_YM         2939 non-null   int64  
 1   DATE_TIME       2939 non-null   object 
 2   WRITER          2939 non-null   object 
 3   SUBCATEGORY     2939 non-null   object 
 4   TITLE           2939 non-null   object 
 5   IMPORTANCE      2939 non-null   float64
 6   ITEM_NAME       1597 non-null   object 
 7   TAG_LIST        2257 non-null   object 
 8   PARSED CONTENT  2939 non-null   object 
dtypes: float64(1), int64(1), object(7)
memory usage: 229.6+ KB


# 중복된 행 제거 

In [118]:
filtered_df = filtered_df.drop_duplicates(subset=['WRITER', 'DATE_TIME', 'TITLE'])

In [119]:
filtered_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2939 entries, 4 to 16166
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   NEWS_YM         2939 non-null   int64  
 1   DATE_TIME       2939 non-null   object 
 2   WRITER          2939 non-null   object 
 3   SUBCATEGORY     2939 non-null   object 
 4   TITLE           2939 non-null   object 
 5   IMPORTANCE      2939 non-null   float64
 6   ITEM_NAME       1597 non-null   object 
 7   TAG_LIST        2257 non-null   object 
 8   PARSED CONTENT  2939 non-null   object 
dtypes: float64(1), int64(1), object(7)
memory usage: 229.6+ KB


# 파일 저장 

In [120]:
# filtered_df 엑셀 파일로 저장 
file_path = '../../../data/News/news_preprocessed.xlsx'  
filtered_df.to_excel(file_path, index=False)

*****파일명: news_preprocessed.xlsx*****